# 📋 GH 주택청약 API 통합 가이드

## 📌 개요

**목적**: 경기주택도시공사(GH) 주택청약 API를 활용한 공고 데이터 수집 및 앱/웹 연동

**API 정보**:
- **이름**: 경기주택도시공사_GH주택청약 모집정보
- **등록일**: 2025-08-21
- **마지막 수정일**: 2025-08-28
- **최신 공고**: 2025-07-31 (광주역세권 청년혁신타운)

**포함 데이터**:
- ✅ 행복주택
- ✅ 국민임대주택
- ✅ 통합공공임대주택
- ✅ 장기전세주택

---

## ⚠️ 주요 제한사항

### 1️⃣ API 업데이트 지연
```
API 최신 데이터:    2025-07-31
홈페이지 최신 데이터: 2025-10-24
차이:               약 85일 (3개월)
```

### 2️⃣ 해결 방안
- **방법 1**: API 데이터 활용 + 홈페이지 링크 제공 (권장)
- **방법 2**: 웹 스크래핑 병행 (고급)
- **방법 3**: API 업데이트 대기

---

## 🎯 이 문서의 목적

1. ✅ GH API 호출 함수 정리
2. ✅ 실제 데이터 출력 예시
3. ✅ 최신 데이터 순 조회 방법
4. ✅ 2025 Endpoint 활용
5. ✅ **LH API 통합 대비** - Metadata 구조 비교 및 통합 포맷 설계

---

## 📚 목차

1. [API 기본 설정](#1-api-기본-설정)
2. [GH API 호출 함수](#2-gh-api-호출-함수)
3. [최신 데이터 조회 함수](#3-최신-데이터-조회-함수)
4. [데이터 내보내기 함수](#4-데이터-내보내기-함수)
5. [실제 사용 예시](#5-실제-사용-예시)
6. [API Metadata 구조](#6-api-metadata-구조)
7. [LH API 통합 준비](#7-lh-api-통합-준비)
8. [최종 권장사항](#8-최종-권장사항)

# 1. API 기본 설정

## 📦 필수 라이브러리

```python
import requests
import pandas as pd
import json
from datetime import datetime
import os
```

## 🔑 API 인증 정보

**공공데이터 포털에서 발급받은 API 키를 사용합니다.**

```python
API_KEY = "your_api_key_here"  # 실제 키로 교체 필요
```

## 🌐 API 엔드포인트

GH 주택청약 API는 3개의 엔드포인트를 제공합니다:

| 엔드포인트 | UDDI | 생성일 | 설명 |
|-----------|------|--------|------|
| 2025 | `uddi:d22eef31-f232-464a-9547-dbff71668860` | 2025-08-21 | 최신 버전 (권장) |
| 2024 | `uddi:e8952a8f-35a9-4f5e-9128-babaf5fec1f4` | 2024-08-21 | 이전 버전 |
| 2023 | `uddi:7e9fa983-2038-4557-bb89-c3d8ce35593c` | 2023-08-21 | 구버전 |

**✅ 권장**: 2025 엔드포인트 사용 (가장 최신 데이터 포함)

**⚠️ 주의**: 3개 엔드포인트 모두 동일한 데이터 소스 사용 (병합 불필요)


In [ ]:
# 필수 라이브러리 import
import requests
import pandas as pd
import json
from datetime import datetime
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# API 설정 (.env 파일에서 읽기)
API_KEY = os.getenv('GH_API_KEY')
if not API_KEY:
    raise ValueError("⚠️  .env 파일에 GH_API_KEY가 설정되지 않았습니다!")

BASE_URL = "https://api.odcloud.kr/api/15119414/v1"

# 엔드포인트 설정
ENDPOINTS = {
    '2025': f"{BASE_URL}/uddi:d22eef31-f232-464a-9547-dbff71668860",
    '2024': f"{BASE_URL}/uddi:e8952a8f-35a9-4f5e-9128-babaf5fec1f4",
    '2023': f"{BASE_URL}/uddi:7e9fa983-2038-4557-bb89-c3d8ce35593c"
}

# 기본 엔드포인트 (2025 권장)
ENDPOINT_2025 = ENDPOINTS['2025']

print("✅ API 설정 완료!")
print(f"📡 BASE URL: {BASE_URL}")
print(f"🔑 API KEY: {API_KEY[:20]}...")
print(f"🌐 사용 엔드포인트: 2025 (최신)")


✅ API 설정 완료!
📡 BASE URL: https://api.odcloud.kr/api/15119414/v1
🔑 API KEY: 7c106f53a255f1d98373...
🌐 사용 엔드포인트: 2025 (최신)


# 2. GH API 호출 함수

## 📡 기본 API 호출 함수

**`get_gh_data(page, per_page, year)`**

### 기능
- GH 주택청약 API에서 페이지별 데이터 조회
- 페이지네이션 지원 (page, per_page)
- 에러 핸들링 포함

### 파라미터
- `page` (int): 페이지 번호 (기본값: 1)
- `per_page` (int): 페이지당 데이터 수 (기본값: 10, 최대 100)
- `year` (str): 엔드포인트 버전 ('2025', '2024', '2023')

### 반환값
```python
{
    'currentCount': 10,
    'data': [...],  # 공고 데이터 리스트
    'matchCount': 990,
    'page': 1,
    'perPage': 10,
    'totalCount': 990
}
```

### 주요 기능
1. ✅ URL 파라미터 인코딩
2. ✅ HTTP 상태 코드 확인
3. ✅ 에러 핸들링 (타임아웃, 네트워크 오류)
4. ✅ 디버깅 정보 출력 (선택적)


In [2]:
def get_gh_data(page=1, per_page=10, year='2025', debug=False):
    """
    GH 주택청약 API 데이터 조회
    
    Parameters:
        page (int): 페이지 번호 (기본값: 1)
        per_page (int): 페이지당 데이터 수 (기본값: 10, 최대: 100)
        year (str): 엔드포인트 버전 ('2025', '2024', '2023')
        debug (bool): 디버깅 정보 출력 여부 (기본값: False)
    
    Returns:
        dict: API 응답 데이터 (실패 시 None)
    """
    # 엔드포인트 선택
    endpoint = ENDPOINTS.get(year, ENDPOINT_2025)
    
    # 요청 파라미터
    params = {
        'page': page,
        'perPage': per_page,
        'serviceKey': API_KEY
    }
    
    try:
        if debug:
            # 디버깅용 URL 출력 (API 키 일부만 표시)
            debug_url = f"{endpoint}?serviceKey={API_KEY[:8]}..."
            print(f"📡 요청 URL: {debug_url}")
        
        # API 호출
        response = requests.get(endpoint, params=params, timeout=10)
        
        if debug:
            print(f"📊 상태 코드: {response.status_code}")
        
        # 상태 코드 확인
        if response.status_code == 200:
            data = response.json()
            
            if debug:
                print(f"✅ 데이터 수신: {data.get('currentCount', 0)}건")
            
            return data
        else:
            print(f"❌ API 오류: {response.status_code}")
            print(f"   응답: {response.text[:200]}")
            return None
            
    except requests.exceptions.Timeout:
        print("⏱️  요청 시간 초과 (10초)")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ 네트워크 오류: {str(e)[:100]}")
        return None
    except Exception as e:
        print(f"❌ 예외 발생: {str(e)[:100]}")
        return None

print("✅ 함수 정의 완료: get_gh_data()")


✅ 함수 정의 완료: get_gh_data()


## 🧪 API 호출 테스트

### ⚠️ 중요: 실행 순서

**반드시 아래 순서대로 실행하세요:**
1. **Cell 2**: API 설정 (API_KEY, ENDPOINTS)
2. **Cell 4**: get_gh_data 함수 정의
3. **Cell 6**: API 데이터 구조 분석

### Step 0: 환경 진단

먼저 API 호출에 필요한 환경이 올바르게 설정되었는지 확인합니다.


In [3]:
# 🔍 Step 0: 환경 진단
print("="*80)
print("🔍 API 호출 환경 진단")
print("="*80)
print()

# 1. 변수 존재 여부 확인
print("📋 1. 필수 변수 확인")
print("─"*80)

check_items = [
    ('API_KEY', 'API_KEY'),
    ('BASE_URL', 'BASE_URL'),
    ('ENDPOINTS', 'ENDPOINTS'),
    ('get_gh_data', 'get_gh_data 함수')
]

all_ok = True

for var_name, display_name in check_items:
    try:
        # 변수 존재 확인
        eval(var_name)
        print(f"  ✅ {display_name}: 정의됨")
    except NameError:
        print(f"  ❌ {display_name}: 정의되지 않음")
        all_ok = False

print()

if not all_ok:
    print("="*80)
    print("⚠️  필수 변수가 정의되지 않았습니다!")
    print("="*80)
    print()
    print("🔧 해결 방법:")
    print("  1. 상단의 Cell 2를 먼저 실행하세요 (API 설정)")
    print("  2. Cell 4를 실행하세요 (get_gh_data 함수 정의)")
    print("  3. 이 셀을 다시 실행하세요")
    print()
else:
    print("="*80)
    print("📡 2. API 연결 테스트")
    print("="*80)
    print()
    
    # 2. 간단한 API 호출 테스트
    print("📡 테스트 API 호출 중 (1개 데이터)...")
    test_result = get_gh_data(page=1, per_page=1, year='2025', debug=True)
    
    print()
    
    if test_result and 'data' in test_result:
        print("="*80)
        print("✅ API 연결 성공!")
        print("="*80)
        print()
        print(f"  • 전체 데이터: {test_result.get('totalCount', 0)}건")
        print(f"  • 응답 시간: 정상")
        print(f"  • 데이터 형식: JSON")
        print()
        print("🎉 모든 환경이 정상입니다. Step 1을 진행하세요!")
    else:
        print("="*80)
        print("❌ API 연결 실패")
        print("="*80)
        print()
        print("🔍 가능한 원인:")
        print()
        print("1️⃣ API 키 문제")
        print("   - 공공데이터 포털에서 활용신청 승인 확인")
        print("   - API 키가 '일반 인증키 (Decoding)' 인지 확인")
        print(f"   - 현재 키: {API_KEY[:20]}...")
        print()
        print("2️⃣ 네트워크 문제")
        print("   - 인터넷 연결 확인")
        print("   - 방화벽/프록시 설정 확인")
        print("   - VPN 비활성화 후 재시도")
        print()
        print("3️⃣ API 서버 문제")
        print("   - 공공데이터 포털 점검 시간 확인")
        print("   - 잠시 후 다시 시도")

print()
print("="*80)


🔍 API 호출 환경 진단

📋 1. 필수 변수 확인
────────────────────────────────────────────────────────────────────────────────
  ✅ API_KEY: 정의됨
  ✅ BASE_URL: 정의됨
  ✅ ENDPOINTS: 정의됨
  ✅ get_gh_data 함수: 정의됨

📡 2. API 연결 테스트

📡 테스트 API 호출 중 (1개 데이터)...
📡 요청 URL: https://api.odcloud.kr/api/15119414/v1/uddi:d22eef31-f232-464a-9547-dbff71668860?serviceKey=7c106f53...
📊 상태 코드: 200
✅ 데이터 수신: 1건

✅ API 연결 성공!

  • 전체 데이터: 990건
  • 응답 시간: 정상
  • 데이터 형식: JSON

🎉 모든 환경이 정상입니다. Step 1을 진행하세요!



### Step 1: 전체 컬럼 구조 파악

API에서 반환되는 모든 필드(컬럼)를 카테고리별로 확인합니다.


In [4]:
# 🔍 Step 1: API 데이터 구조 분석
print("="*80)
print("🔍 API 데이터 구조 분석")
print("="*80)
print()

# API 호출 (1개 데이터만)
print("📡 API 호출 중...")
result = get_gh_data(page=1, per_page=1, year='2025', debug=False)

if result and 'data' in result and len(result['data']) > 0:
    print("✅ API 호출 성공!\n")
    
    # 첫 번째 공고 데이터
    first_notice = result['data'][0]
    
    # 전체 컬럼(필드) 목록
    all_columns = list(first_notice.keys())
    
    print("="*80)
    print(f"📋 반환되는 전체 컬럼 목록 (총 {len(all_columns)}개)")
    print("="*80)
    print()
    
    # 컬럼을 카테고리별로 분류
    categories = {
        '🔢 기본 정보': [],
        '📍 위치 정보': [],
        '📆 일정 정보': [],
        '🏠 주택 정보': [],
        '📎 첨부 정보': [],
        '📝 기타 정보': []
    }
    
    # 컬럼 분류
    for col in all_columns:
        if any(keyword in col for keyword in ['공고', '게시', '구분', '모집', '번호']):
            categories['🔢 기본 정보'].append(col)
        elif any(keyword in col for keyword in ['주소', '우편', '시군구', '전화', '지도']):
            categories['📍 위치 정보'].append(col)
        elif any(keyword in col for keyword in ['일자', '시각', '예정', '시작', '종료', '기간']):
            categories['📆 일정 정보'].append(col)
        elif any(keyword in col for keyword in ['주택', '사업', '입주']):
            categories['🏠 주택 정보'].append(col)
        elif any(keyword in col for keyword in ['첨부', '파일']):
            categories['📎 첨부 정보'].append(col)
        else:
            categories['📝 기타 정보'].append(col)
    
    # 카테고리별 출력
    for category, columns in categories.items():
        if columns:
            print(f"{category} ({len(columns)}개)")
            print("─"*80)
            for i, col in enumerate(columns, 1):
                value = first_notice.get(col, 'N/A')
                
                # 값이 너무 길면 축약
                if isinstance(value, str) and len(value) > 50:
                    value = value[:50] + "..."
                
                print(f"  [{i:2d}] {col:30} : {value}")
            print()
    
    print("="*80)
    print("💾 변수 저장: first_notice (전체 데이터)")
    print("="*80)
    print()
    print("💡 사용법:")
    print("   - first_notice.keys() : 모든 컬럼명 확인")
    print("   - first_notice['컬럼명'] : 특정 컬럼 값 확인")
    print("   - pd.DataFrame([first_notice]) : DataFrame으로 변환")
    
else:
    print("❌ API 호출 실패")
    print()
    print("🔍 문제 해결:")
    print("  1. API_KEY가 올바른지 확인")
    print("  2. 인터넷 연결 확인")
    print("  3. 공공데이터 포털에서 API 활용 신청 승인 확인")
    print("  4. 서비스 키 디코딩 여부 확인 (Decoding/Encoding)")

print()
print("="*80)


🔍 API 데이터 구조 분석

📡 API 호출 중...
✅ API 호출 성공!

📋 반환되는 전체 컬럼 목록 (총 62개)

🔢 기본 정보 (18개)
────────────────────────────────────────────────────────────────────────────────
  [ 1] 게시시각                           : None
  [ 2] 게시일자                           : 2017-11-30
  [ 3] 결과게시시작시간                       : None
  [ 4] 결과게시시작일자                       : None
  [ 5] 결과게시종료시간                       : None
  [ 6] 결과게시종료일자                       : None
  [ 7] 공고명                            : 수원영통 경기도형 행복주택 입주자 모집
  [ 8] 공고번호                           : 9
  [ 9] 공고시각                           : 15:00
  [10] 공고일자                           : 2017-11-30
  [11] 구분                             : 1
  [12] 모집횟수                           : 1
  [13] 문의처전화번호                        : None
  [14] 접수처우편번호                        : 16686
  [15] 접수처전화번호                        : None
  [16] 제출서류첨부파일일련번호                   : None
  [17] 주택관리번호                         : 2017001669
  [18] 첨부파일일련번호                       : 30

### Step 2: API 호출 테스트 (10개 데이터)

API에서 10개의 공고 데이터를 조회하고 주요 필드를 확인합니다.


In [5]:
# 🧪 Step 2: API 호출 테스트 (10개 데이터)
print("="*80)
print("🧪 API 호출 테스트 (10개 데이터)")
print("="*80)
print()

# API 호출
print("📡 API 호출 중...")
result = get_gh_data(page=1, per_page=10, year='2025', debug=True)

if result:
    print()
    print("="*80)
    print("📊 API 응답 정보")
    print("="*80)
    print()
    print(f"  • 현재 페이지: {result.get('page')}페이지")
    print(f"  • 페이지당 데이터: {result.get('perPage')}건")
    print(f"  • 현재 페이지 데이터: {result.get('currentCount')}건")
    print(f"  • 전체 데이터: {result.get('totalCount')}건")
    print()
    
    # 데이터 확인
    if 'data' in result and len(result['data']) > 0:
        print("="*80)
        print("📢 조회된 공고 목록 (주요 필드)")
        print("="*80)
        print()
        
        # 주요 필드 정의
        key_fields = {
            '공고번호': '번호',
            '공고명': '제목',
            '공고일자': '공고일',
            '시군구명': '지역',
            '접수시작일자': '접수시작',
            '접수종료일자': '접수종료',
            '구분': '유형'
        }
        
        # 각 공고 출력
        for idx, notice in enumerate(result['data'], 1):
            print(f"[{idx:2d}] ─────────────────────────────────────────────────────────")
            
            for field_name, display_name in key_fields.items():
                value = notice.get(field_name, 'N/A')
                
                # 날짜 포맷팅
                if '일자' in field_name and value != 'N/A' and pd.notna(value):
                    value_str = str(value)
                    if len(value_str) == 8:
                        value = f"{value_str[:4]}-{value_str[4:6]}-{value_str[6:]}"
                
                # 공고명은 길이 제한
                if field_name == '공고명' and isinstance(value, str) and len(value) > 60:
                    value = value[:60] + "..."
                
                print(f"     {display_name:8} : {value}")
            
            print()
        
        print("="*80)
        print("✅ API 호출 성공!")
        print("="*80)
        print()
        
        # DataFrame으로 변환
        df_result = pd.DataFrame(result['data'])
        
        print("💾 변수 저장:")
        print(f"  • result : API 응답 전체 (dict)")
        print(f"  • df_result : 공고 데이터 DataFrame (shape: {df_result.shape})")
        print()
        print("💡 DataFrame 활용:")
        print("  • df_result.head() : 상위 5개 확인")
        print("  • df_result.columns : 모든 컬럼명")
        print("  • df_result['공고명'] : 특정 컬럼 조회")
        print("  • df_result[df_result['시군구명'] == '광주시'] : 필터링")
        
    else:
        print("⚠️  데이터가 없습니다")
        
else:
    print()
    print("="*80)
    print("❌ API 호출 실패")
    print("="*80)
    print()
    print("🔍 문제 해결 방법:")
    print()
    print("1️⃣ **API 키 확인**")
    print("   - 공공데이터 포털(data.go.kr)에서 발급받은 키인지 확인")
    print("   - 활용신청이 승인되었는지 확인")
    print()
    print("2️⃣ **디코딩/인코딩 확인**")
    print("   - 공공데이터 포털에서 '일반 인증키 (Decoding)' 선택")
    print("   - URL 인코딩된 키가 아닌 일반 키 사용")
    print()
    print("3️⃣ **인터넷 연결 확인**")
    print("   - 방화벽 설정 확인")
    print("   - VPN 사용 시 비활성화 후 재시도")
    print()
    print("4️⃣ **엔드포인트 확인**")
    print(f"   - 현재 엔드포인트: {ENDPOINTS['2025']}")
    print("   - 공공데이터 포털에서 URL 변경 여부 확인")

print()
print("="*80)


🧪 API 호출 테스트 (10개 데이터)

📡 API 호출 중...
📡 요청 URL: https://api.odcloud.kr/api/15119414/v1/uddi:d22eef31-f232-464a-9547-dbff71668860?serviceKey=7c106f53...
📊 상태 코드: 200
✅ 데이터 수신: 10건

📊 API 응답 정보

  • 현재 페이지: 1페이지
  • 페이지당 데이터: 10건
  • 현재 페이지 데이터: 10건
  • 전체 데이터: 990건

📢 조회된 공고 목록 (주요 필드)

[ 1] ─────────────────────────────────────────────────────────
     번호       : 9
     제목       : 수원영통 경기도형 행복주택 입주자 모집
     공고일      : 2017-11-30
     지역       : N/A
     접수시작     : 2017-12-06
     접수종료     : 2017-12-15
     유형       : 1

[ 2] ─────────────────────────────────────────────────────────
     번호       : 10
     제목       : 남양주 다산역(A2) 경기도형 행복주택 입주자모집
     공고일      : 2017-11-30
     지역       : N/A
     접수시작     : 2017-12-06
     접수종료     : 2017-12-15
     유형       : 2

[ 3] ─────────────────────────────────────────────────────────
     번호       : 14
     제목       : 가평청사복합 경기행복주택 입주자 모집
     공고일      : 2018-08-30
     지역       : N/A
     접수시작     : 2018-09-05
     접수종료     : 2018-09-14
     유형 

# 3. 최신 데이터 조회 함수

## 중요 문제점

API는 등록순으로 데이터를 반환합니다 (오래된순)

따라서 대량 데이터를 수집한 후 정렬이 필요합니다.


In [6]:
def get_latest_notices_for_app(count=10, exclude_test=True, min_year='2024'):
    """
    앱/웹 연동을 위한 최신 공고 조회 함수
    
    Parameters:
        count (int): 반환할 공고 개수
        exclude_test (bool): 테스트 데이터 제외 여부
        min_year (str): 최소 연도
    
    Returns:
        dict: 최신 공고 데이터 및 메타데이터
    """
    print(f"📡 API 데이터 수집 중 (최소 연도: {min_year})...")
    
    # 1. API 전체 데이터 수집
    all_notices = []
    page = 1
    
    while True:
        result = get_gh_data(page=page, per_page=100, year='2025')
        
        if result and 'data' in result and len(result['data']) > 0:
            all_notices.extend(result['data'])
            
            if len(result['data']) < 100:
                break
            
            page += 1
            
            if page > 15:  # 최대 1500개까지만
                break
        else:
            break
    
    if not all_notices:
        return {
            'success': False,
            'error': 'API 데이터 수집 실패',
            'count': 0,
            'data': []
        }
    
    print(f"✅ 총 {len(all_notices)}건 수집 완료\\n")
    
    # 2. DataFrame으로 변환
    df_all = pd.DataFrame(all_notices)
    
    # 3. 데이터 필터링
    if '공고일자' not in df_all.columns:
        return {
            'success': False,
            'error': '공고일자 컬럼이 없습니다',
            'count': 0,
            'data': []
        }
    
    # None 값 제거
    df_filtered = df_all[df_all['공고일자'].notna()].copy()
    
    # 테스트 데이터 제외
    if exclude_test:
        df_filtered = df_filtered[
            ~df_filtered['공고명'].str.contains('테스트|test|Test', case=False, na=False)
        ].copy()
    
    # 날짜 문자열로 변환
    df_filtered['공고일자_str'] = df_filtered['공고일자'].astype(str)
    
    # 지정된 연도 이후만 필터
    min_date = f"{min_year}0101"
    df_filtered = df_filtered[df_filtered['공고일자_str'] >= min_date].copy()
    
    if len(df_filtered) == 0:
        return {
            'success': False,
            'error': f'{min_year}년 이후 데이터가 없습니다',
            'count': 0,
            'data': []
        }
    
    # 4. 날짜순 정렬 (최신순)
    df_sorted = df_filtered.sort_values('공고일자', ascending=False)
    
    # 5. 최신 N개만 추출
    df_result = df_sorted.head(count).copy()
    
    # 6. API 최신 업데이트 날짜 확인
    latest_date = df_result.iloc[0]['공고일자_str']
    latest_date_formatted = f"{latest_date[:4]}-{latest_date[4:6]}-{latest_date[6:]}"
    
    # 7. 안내 메시지 생성
    notice_message = f"본 데이터는 {latest_date_formatted} 기준입니다. 최신 공고는 GH 홈페이지를 확인하세요."
    
    # 8. 결과 반환
    print(f"🎯 최신 {len(df_result)}개 추출 완료")
    print(f"📅 API 마지막 업데이트: {latest_date_formatted}")
    print(f"💡 안내: {notice_message}\\n")
    
    return {
        'success': True,
        'last_api_update': latest_date_formatted,
        'notice': notice_message,
        'count': len(df_result),
        'data': df_result.to_dict('records'),
        'website_link': 'https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseList.do',
        'metadata': {
            'total_collected': len(all_notices),
            'after_filter': len(df_filtered),
            'returned': len(df_result),
            'min_year': min_year,
            'exclude_test': exclude_test
        }
    }

print("✅ 함수 정의 완료: get_latest_notices_for_app()")


✅ 함수 정의 완료: get_latest_notices_for_app()


## 4. 데이터 내보내기 함수
### JSON 내보내기 코드

In [7]:
# JSON 내보내기 코드

def export_latest_notices_json(result_dict, filename=None):
    """
    API 결과를 JSON 파일로 내보내기
    
    Parameters:
        result_dict (dict): get_latest_notices_for_app() 결과
        filename (str): 저장할 파일명 (None이면 자동 생성)
    
    Returns:
        str: 저장된 파일 경로
    """
    if not result_dict.get('success'):
        print("❌ 내보낼 데이터가 없습니다")
        return None
    
    # 파일명 자동 생성
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"gh_notices_{timestamp}.json"
    
    # 디렉토리 생성
    os.makedirs('output', exist_ok=True)
    filepath = os.path.join('output', filename)
    
    # JSON 저장
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(result_dict, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 파일 저장 완료!")
    print(f"📁 경로: {filepath}")
    print(f"📊 데이터: {result_dict['count']}건")
    
    return filepath

print("✅ 함수 정의 완료: export_latest_notices_json()")

✅ 함수 정의 완료: export_latest_notices_json()


# 9. 전체 데이터 수집 및 CSV 저장 (POST 요청 정보 포함)

## 🎯 **최종 목표**

1. ✅ **API 제공 공고** (2024년~2025-07-31) 수집
2. ✅ **중복 제거** 및 **테스트 공고 제외**
3. ✅ **POST 요청 정보** 추가 (크롤링용)
4. ✅ **CSV 파일** 저장 (팀원 공유)

---

## 📊 **중요 발견사항**

### ✅ **POST 요청 확인 (Chrome 개발자 도구)**

```
Request Method: POST
Content-Type: application/x-www-form-urlencoded

Form Data:
  pbancNo: 742          ← 공고번호
  pbancKndCd: 01        ← 공고 종류 코드
  bizTyNm: 통합공공임대   ← 사업 유형명
  gvPgmId: SR7150M00    ← 프로그램 ID (고정)
```

### ⚠️ **API 제한사항**

- **API 제공**: 2025-07-31까지의 공고
- **API 미제공**: 2025-08-01 이후 공고 (최신 10개)
  - 공고번호: 762, 761, 757, 755, 754, 753, 743, 747, 745, 744

---


In [14]:
# GH 홈페이지 URL 정보 (POST 요청 기반)
print("="*80)
print("📋 POST 요청 정보 (Chrome 개발자 도구에서 확인 완료)")
print("="*80)
print()

# GH 홈페이지 기본 정보
GH_BASE_URL = "https://apply.gh.or.kr"
GH_LIST_URL = f"{GH_BASE_URL}/sb/sr/sr7150/selectPbancRentHouseList.do"
GH_DETAIL_URL = f"{GH_BASE_URL}/sb/sr/sr7150/selectPbancDetailView.do"

print("✅ 확인된 URL:")
print(f"  • 목록 페이지: {GH_LIST_URL}")
print(f"  • 상세 페이지 (POST): {GH_DETAIL_URL}")
print()

print("✅ POST 요청 Form Data 예시:")
print("─"*80)
example_data = {
    'searchArea': '',
    'searchCate': '',
    'searchState': '',
    'searchTitle': '',
    'previewYn': 'N',
    'pbancNo': '742',
    'pbancKndCd': '01',
    'bizTyNm': '통합공공임대',
    'pageIndex': '1',
    'gvPgmId': 'SR7150M00'
}

for key, value in example_data.items():
    print(f"  {key:15s}: {value}")
print()

print("✅ 매핑 규칙:")
print("─"*80)
print("  • pbancNo: API '공고번호' → 그대로 사용")
print("  • pbancKndCd: API '구분' → 2자리 문자열 ('01', '02', ...)")
print("  • bizTyNm: API '공고명' → 키워드 추출 (행복주택, 국민임대, ...)")
print()

print("="*80)


📋 POST 요청 정보 (Chrome 개발자 도구에서 확인 완료)

✅ 확인된 URL:
  • 목록 페이지: https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseList.do
  • 상세 페이지 (POST): https://apply.gh.or.kr/sb/sr/sr7150/selectPbancDetailView.do

✅ POST 요청 Form Data 예시:
────────────────────────────────────────────────────────────────────────────────
  searchArea     : 
  searchCate     : 
  searchState    : 
  searchTitle    : 
  previewYn      : N
  pbancNo        : 742
  pbancKndCd     : 01
  bizTyNm        : 통합공공임대
  pageIndex      : 1
  gvPgmId        : SR7150M00

✅ 매핑 규칙:
────────────────────────────────────────────────────────────────────────────────
  • pbancNo: API '공고번호' → 그대로 사용
  • pbancKndCd: API '구분' → 2자리 문자열 ('01', '02', ...)
  • bizTyNm: API '공고명' → 키워드 추출 (행복주택, 국민임대, ...)



## 🔍 POST 요청 확인됨!

**개발자 도구 확인 결과:**
- Request Method: **POST**
- Content-Type: `application/x-www-form-urlencoded`

→ URL만으로는 특정 공고에 접근 불가!

### 📋 추가 확인 필요: Form Data (Payload)

**다시 개발자 도구에서 확인:**

1. F12 → Network 탭
2. 공고 클릭
3. `selectPbancDetailView.do` 요청 선택
4. **Payload** 탭 클릭 ← 중요!

**Form Data 예시:**
```
pbancNo: 742
menuId: xxx
searchType: xxx
...
```

이 정보가 있어야 POST 요청을 재현할 수 있습니다!


## 📋 Step 1-1: 목록 페이지에서 실제 링크 추출

BeautifulSoup을 사용하여 목록 페이지의 HTML을 분석하고 실제 링크 패턴을 찾습니다.


## 📋 방법 2 실행: HTML에서 POST Form Data 패턴 찾기

POST 요청이므로 HTML에서 Form 또는 JavaScript 함수를 분석합니다.


In [15]:
# 방법 2: HTML 분석으로 POST Form Data 패턴 찾기
print("="*80)
print("🔍 목록 페이지 HTML 상세 분석 (POST Form 찾기)")
print("="*80)
print()

try:
    from bs4 import BeautifulSoup
    import re
    
    print("📡 목록 페이지 접속 중...")
    print(f"URL: {GH_LIST_URL}\n")
    
    response = requests.get(GH_LIST_URL, timeout=15)
    
    if response.status_code == 200:
        print("✅ 페이지 로드 성공!\n")
        
        # HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # ===== 1. 테이블에서 공고 목록 찾기 =====
        print("="*80)
        print("📋 1단계: 공고 목록 테이블 분석")
        print("="*80)
        print()
        
        # 공고 목록은 보통 <table> 또는 <ul> 안에 있음
        tables = soup.find_all('table')
        
        if tables:
            print(f"✅ {len(tables)}개의 테이블 발견")
            print()
            
            # 가장 큰 테이블이 공고 목록일 가능성 높음
            for idx, table in enumerate(tables[:3], 1):
                rows = table.find_all('tr')
                print(f"  테이블 {idx}: {len(rows)}개 행")
                
                # 각 행의 첫 번째 링크 확인
                if len(rows) > 1:  # 헤더 제외
                    first_data_row = rows[1] if len(rows) > 1 else rows[0]
                    links = first_data_row.find_all('a')
                    
                    if links:
                        first_link = links[0]
                        onclick = first_link.get('onclick', '')
                        href = first_link.get('href', '')
                        text = first_link.get_text(strip=True)[:50]
                        
                        print(f"    첫 번째 링크 텍스트: {text}")
                        if onclick:
                            print(f"    onClick: {onclick[:100]}")
                        if href and href != '#':
                            print(f"    href: {href[:100]}")
                print()
        
        # ===== 2. onClick 이벤트에서 함수 호출 패턴 찾기 =====
        print("="*80)
        print("📋 2단계: onClick 이벤트 분석 (핵심!)")
        print("="*80)
        print()
        
        all_links = soup.find_all('a')
        onclick_patterns = {}
        
        for link in all_links:
            onclick = link.get('onclick', '')
            if onclick:
                # 함수 이름과 파라미터 추출
                # 예: goDetail('742', 'menuId') 또는 fn_detail('742')
                match = re.search(r'(\w+)\s*\(([^)]+)\)', onclick)
                if match:
                    func_name = match.group(1)
                    params = match.group(2)
                    
                    if func_name not in onclick_patterns:
                        onclick_patterns[func_name] = []
                    onclick_patterns[func_name].append(params)
        
        if onclick_patterns:
            print("✅ onClick 함수 발견:")
            print()
            for func_name, param_list in onclick_patterns.items():
                print(f"  📌 함수명: {func_name}")
                print(f"     호출 횟수: {len(param_list)}회")
                print(f"     파라미터 예시:")
                for i, params in enumerate(param_list[:3], 1):
                    print(f"       [{i}] {params}")
                print()
        else:
            print("  ⚠️  onClick 패턴을 찾지 못했습니다")
            print()
        
        # ===== 3. JavaScript 함수 정의 찾기 =====
        print("="*80)
        print("📋 3단계: JavaScript 함수 정의 찾기")
        print("="*80)
        print()
        
        scripts = soup.find_all('script')
        found_functions = {}
        
        for script in scripts:
            if script.string:
                script_text = script.string
                
                # onClick에서 발견한 함수들의 정의 찾기
                for func_name in onclick_patterns.keys():
                    # function 함수명( 패턴 찾기
                    pattern = rf'function\s+{func_name}\s*\([^)]*\)\s*\{{'
                    match = re.search(pattern, script_text)
                    
                    if match:
                        # 함수 시작 위치
                        start = match.start()
                        # 함수 전체 추출 (처음 500자만)
                        func_body = script_text[start:start+500]
                        found_functions[func_name] = func_body
        
        if found_functions:
            print("✅ JavaScript 함수 정의 발견:")
            print()
            for func_name, func_body in found_functions.items():
                print(f"  📌 함수: {func_name}")
                print("  " + "─"*76)
                # 줄바꿈 유지하며 출력
                for line in func_body.split('\n')[:15]:  # 처음 15줄만
                    print(f"  {line}")
                print(f"  ... (생략)")
                print()
        else:
            print("  ⚠️  함수 정의를 찾지 못했습니다")
            print()
        
        # ===== 4. Form 요소 찾기 =====
        print("="*80)
        print("📋 4단계: Form 요소 분석")
        print("="*80)
        print()
        
        forms = soup.find_all('form')
        
        if forms:
            print(f"✅ {len(forms)}개의 Form 발견:")
            print()
            for idx, form in enumerate(forms, 1):
                form_id = form.get('id', 'N/A')
                form_name = form.get('name', 'N/A')
                action = form.get('action', 'N/A')
                method = form.get('method', 'GET').upper()
                
                print(f"  [{idx}] Form")
                print(f"      ID: {form_id}")
                print(f"      Name: {form_name}")
                print(f"      Action: {action}")
                print(f"      Method: {method}")
                
                # Form 내부의 input 필드 확인
                inputs = form.find_all('input')
                if inputs:
                    print(f"      Input 필드: {len(inputs)}개")
                    for input_tag in inputs[:5]:  # 처음 5개만
                        input_name = input_tag.get('name', 'N/A')
                        input_type = input_tag.get('type', 'text')
                        print(f"        • {input_name} ({input_type})")
                print()
        else:
            print("  ⚠️  Form을 찾지 못했습니다")
            print()
        
        # ===== 5. 결론 및 권장사항 =====
        print("="*80)
        print("💡 분석 결과 종합")
        print("="*80)
        print()
        
        if onclick_patterns and found_functions:
            print("✅ POST 요청 패턴 발견 가능성 높음!")
            print()
            print("📝 다음 단계:")
            print("  1. 위의 JavaScript 함수에서 POST 요청 코드 확인")
            print("  2. Form Data로 전송되는 파라미터 확인")
            print("  3. 또는 개발자 도구의 'Payload' 탭에서 직접 확인")
            print()
            print("🔍 개발자 도구에서 확인:")
            print("  F12 → Network → 공고 클릭 → selectPbancDetailView.do")
            print("  → Payload 탭 → Form Data 확인")
            print()
        elif onclick_patterns:
            print("⚠️  onClick 함수는 발견했으나 정의를 찾지 못함")
            print()
            print("📝 대안:")
            print("  • 외부 JavaScript 파일 확인 필요")
            print("  • 또는 개발자 도구로 직접 Payload 확인")
            print()
        else:
            print("❌ onClick 패턴을 찾지 못했습니다")
            print()
            print("📝 가능한 이유:")
            print("  • JavaScript로 동적 렌더링")
            print("  • 이벤트 리스너 사용 (addEventListener)")
            print("  • React/Vue 등 프레임워크 사용")
            print()
            print("📝 해결 방법:")
            print("  → 개발자 도구의 Payload 탭 확인 (가장 확실)")
            print()
        
        print("="*80)
        print("🎯 최종 권장사항")
        print("="*80)
        print()
        print("POST 요청이므로 URL만으로는 직접 접근 불가!")
        print()
        print("✅ 최선의 방법:")
        print("  1. 개발자 도구 → Network → Payload 탭")
        print("  2. Form Data에서 pbancNo 등의 파라미터 확인")
        print("  3. Python requests로 POST 요청 재현")
        print()
        print("✅ 대안 (크롤링용):")
        print("  • CSV에 목록 페이지 URL만 제공")
        print("  • Selenium으로 공고번호 검색 → 클릭 → 크롤링")
        print()
        
    else:
        print(f"❌ 페이지 로드 실패: {response.status_code}")

except ImportError:
    print("❌ BeautifulSoup4가 설치되지 않았습니다")
    print()
    print("설치 명령:")
    print("  pip install beautifulsoup4")

except Exception as e:
    print(f"❌ 오류 발생: {str(e)}")
    import traceback
    traceback.print_exc()

print()
print("="*80)


🔍 목록 페이지 HTML 상세 분석 (POST Form 찾기)

📡 목록 페이지 접속 중...
URL: https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseList.do

✅ 페이지 로드 성공!

📋 1단계: 공고 목록 테이블 분석

✅ 1개의 테이블 발견

  테이블 1: 11개 행
    첫 번째 링크 텍스트: 안성공도 참아름 5단지 국민임대주택 예비입주자 모집공고
    href: #a

📋 2단계: onClick 이벤트 분석 (핵심!)

✅ onClick 함수 발견:

  📌 함수명: fn_sel_page
     호출 횟수: 13회
     파라미터 예시:
       [1] 1
       [2] 1
       [3] 2

📋 3단계: JavaScript 함수 정의 찾기

✅ JavaScript 함수 정의 발견:

  📌 함수: fn_sel_page
  ────────────────────────────────────────────────────────────────────────────
  function fn_sel_page(pageIndex) {
  	$('#pageIndex').val(pageIndex);
  	$net.submit("bbsSearchFrm", "/sb/sr/sr7150/selectPbancRentHouseList.do");
  }
  
  $(function() {
  	
  	$('#search_btn').on('click', function() {
  		search.submit();
  	});
  
  	
  	$('.search_box').on('keyup', function(event) {
  		if (event.keyCode == 13) {
  			event.preventDefault();
  ... (생략)

📋 4단계: Form 요소 분석

✅ 2개의 Form 발견:

  [1] Form
      ID: bbsSearchFrm
      Name: N/A

## 🎯 **가장 중요: Payload (Form Data) 확인**

POST 요청의 실제 데이터를 확인하는 방법입니다.

### 📋 단계별 가이드

**1단계: 개발자 도구 열기**
```
F12 또는 Ctrl+Shift+I
```

**2단계: Network 탭 선택**

**3단계: 공고 하나 클릭**
- 예: "광주역세권 청년혁신타운"

**4단계: `selectPbancDetailView.do` 요청 찾기**

**5단계: ⭐ Payload 탭 클릭 (Headers 옆)**

**6단계: Form Data 확인**

### 예상되는 Form Data 형식:

```
pbancNo: 742
menuId: 01
searchType: 1
pageIndex: 1
... (기타 파라미터)
```

### 📸 확인 후 여기에 복사해주세요!
Form Data 내용을 복사하면 POST 요청을 Python으로 재현할 수 있습니다.
> 결과 1
From Data
- Payload

searchArea :
searchCate :
searchState :
searchTitle :
previewYn : N
pbancNo : 762
pbancKndCd : 01
bizTyNm : 국민임대
pageIndex : 1
gvPgmId : SR7150M00

View source : searchArea=&searchCate=&searchState=&searchTitle=&previewYn=N&pbancNo=762&pbancKndCd=01&bizTyNm=%EA%B5%AD%EB%AF%BC%EC%9E%84%EB%8C%80&pageIndex=1&gvPgmId=SR7150M00

View URL-encoded : %EA%B5%AD%EB%AF%BC%EC%9E%84%EB%8C%80
-> '국민임대' 인코딩된 결과

> 결과 2
searchArea :
searchCate :
searchState :
searchTitle :
previewYn : N
pbancNo : 742
pbancKndCd : 01
bizTyNm : 통합공공임대
pageIndex : 2
gvPgmId : SR7150M00

View source : searchArea=&searchCate=&searchState=&searchTitle=&previewYn=N&pbancNo=742&pbancKndCd=01&bizTyNm=%ED%86%B5%ED%95%A9%EA%B3%B5%EA%B3%B5%EC%9E%84%EB%8C%80&pageIndex=2&gvPgmId=SR7150M00

View URL-encoded : %ED%86%B5%ED%95%A9%EA%B3%B5%EA%B3%B5%EC%9E%84%EB%8C%80
-> '통합공공임대' 인코딩된 결과

---

## 💡 **실용적인 해결책: POST 요청 대응**

POST 요청이므로 URL만으로는 특정 공고에 직접 접근 불가!

### ✅ 권장 방법: 목록 페이지 + Selenium 크롤링

**CSV 구조:**
```
공고번호 | 공고명 | ... | list_url | 공고번호_검색
───────────────────────────────────────────────────────
742 | 광주역세권... | ... | https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseList.do | 742
741 | 안성청사복합... | ... | https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseList.do | 741
```

**크롤링 방법:**
1. 목록 페이지 접속
2. 공고번호로 검색 또는 직접 찾기
3. 해당 공고 클릭
4. 상세 페이지 크롤링

---


In [18]:
# POST 요청 대응: URL 생성 함수 수정
print("="*80)
print("🔧 POST 요청 대응: URL 함수 재정의")
print("="*80)
print()

# GH 홈페이지 기본 URL (재정의)
GH_BASE_URL = "https://apply.gh.or.kr"
GH_LIST_URL = f"{GH_BASE_URL}/sb/sr/sr7150/selectPbancRentHouseList.do"

def generate_list_page_url():
    """
    목록 페이지 URL 반환
    (POST 요청이므로 직접 링크 불가)
    """
    return GH_LIST_URL

def generate_notice_url(notice_number):
    """
    공고번호로 URL 생성 (POST 요청 대응)
    
    Parameters:
        notice_number: 공고번호
    
    Returns:
        str: 목록 페이지 URL (직접 링크 불가능)
    
    Note:
        POST 요청이므로 특정 공고로 직접 이동할 수 없습니다.
        목록 페이지 URL을 반환하며, Selenium으로 크롤링해야 합니다.
    """
    # POST 요청이므로 목록 페이지 URL만 반환
    return GH_LIST_URL

print("✅ 함수 재정의 완료!")
print()
print("📋 변경 사항:")
print("  • generate_notice_url() → 목록 페이지 URL 반환")
print("  • POST 요청이므로 직접 링크 불가")
print("  • Selenium으로 크롤링 필요")
print()
print("💡 CSV 구조:")
print("  공고번호 | 공고명 | ... | url (목록 페이지)")
print()
print("🔧 크롤링 방법:")
print("  1. url 컬럼의 목록 페이지 접속")
print("  2. 공고번호로 검색 또는 찾기")
print("  3. 해당 공고 클릭")
print("  4. 상세 페이지 크롤링")
print()
print("="*80)


🔧 POST 요청 대응: URL 함수 재정의

✅ 함수 재정의 완료!

📋 변경 사항:
  • generate_notice_url() → 목록 페이지 URL 반환
  • POST 요청이므로 직접 링크 불가
  • Selenium으로 크롤링 필요

💡 CSV 구조:
  공고번호 | 공고명 | ... | url (목록 페이지)

🔧 크롤링 방법:
  1. url 컬럼의 목록 페이지 접속
  2. 공고번호로 검색 또는 찾기
  3. 해당 공고 클릭
  4. 상세 페이지 크롤링



## 🎉 **Payload 확인 완료!**

### 📋 발견된 Form Data:
현재 API 호출 x | Web 상 올라와있는 공고 : 안성공도 참아름 5단지 국민임대주택 예비입주자 모집공고

```
searchArea: (빈값)
searchCate: (빈값)
searchState: (빈값)
searchTitle: (빈값)
previewYn: N
pbancNo: 762          ← 공고번호 (핵심!)
pbancKndCd: 01        ← 공고 종류 코드
bizTyNm: 국민임대      ← 사업 유형명
pageIndex: 1
gvPgmId: SR7150M00    ← 프로그램 ID
```

현재 API 호출되는 공고1 : 용인영덕 경기행복주택 입주자 추가모집 공고
```
searchArea: (빈값)
searchCate: (빈값)
searchState: (빈값)
searchTitle: (빈값)
previewYn: N
pbancNo: 736          ← 공고번호 (핵심!)
pbancKndCd: 01        ← 공고 종류 코드
bizTyNm: 행복주택      ← 사업 유형명
pageIndex: 2
gvPgmId: SR7150M00    ← 프로그램 ID
```

현재 API 호출되는 공고2 : (최초) 광주역세권 청년혁신타운 통합공공임대주택(일자리연계형 지원주택) 입주자 모집 공고
```
searchArea: (빈값)
searchCate: (빈값)
searchState: (빈값)
searchTitle: (빈값)
previewYn: N
pbancNo: 742          ← 공고번호 (핵심!)
pbancKndCd: 01        ← 공고 종류 코드
bizTyNm: 통합공공임대  ← 사업 유형명
pageIndex: 2
gvPgmId: SR7150M00    ← 프로그램 ID
```

현재 API 호출되는 공고3 : 반월역 두산위브더센트럴 장기전세 입주자 모집공고
```
searchArea: (빈값)
searchCate: (빈값)
searchState: (빈값)
searchTitle: (빈값)
previewYn: N
pbancNo: 728          ← 공고번호 (핵심!)
pbancKndCd: 01        ← 공고 종류 코드
bizTyNm: 장기전세      ← 사업 유형명
pageIndex: 2
gvPgmId: SR7150M00    ← 프로그램 ID
```

현재 API 호출되는 공고4 : [정정공고]안성공도 참아름 2단지 국민임대주택 예비입주자 모집공고
```
searchArea: (빈값)
searchCate: (빈값)
searchState: (빈값)
searchTitle: (빈값)
previewYn: N
pbancNo: 709          ← 공고번호 (핵심!)
pbancKndCd: 02        ← 공고 종류 코드
bizTyNm: 국민임대      ← 사업 유형명
pageIndex: 3
gvPgmId: SR7150M00    ← 프로그램 ID
```

현재 API 호출되는 공고5 : 다산 센트럴파크6단지(지금A1) 영구임대주택 입주완화 입주자 및 예비자 모집
```
searchArea: (빈값)
searchCate: (빈값)
searchState: (빈값)
searchTitle: (빈값)
previewYn: N
pbancNo: 676          ← 공고번호 (핵심!)
pbancKndCd: 01        ← 공고 종류 코드
bizTyNm: 영구임대      ← 사업 유형명
pageIndex: 5
gvPgmId: SR7150M00    ← 프로그램 ID
```

### 🔑 핵심 파라미터:

1. **`pbancNo`**: 공고번호 (API에서 제공)
2. **`pbancKndCd`**: 공고 종류 코드 (API의 '구분' 필드와 매핑 필요)
3. **`bizTyNm`**: 사업 유형명 (행복주택, 국민임대, 통합공공임대, 장기전세)
4. **`gvPgmId`**: 프로그램 ID (고정값: SR7150M00)

---


In [19]:
# POST 요청으로 공고 상세 페이지 가져오기
print("="*80)
print("🔍 POST 요청 함수 생성")
print("="*80)
print()

import requests
from urllib.parse import quote

def get_notice_detail_by_post(pbanc_no, pbanc_knd_cd='01', biz_ty_nm='국민임대'):
    """
    POST 요청으로 공고 상세 페이지 HTML 가져오기
    
    Parameters:
        pbanc_no: 공고번호
        pbanc_knd_cd: 공고 종류 코드 (기본값: '01')
        biz_ty_nm: 사업 유형명 (기본값: '국민임대')
    
    Returns:
        str: 상세 페이지 HTML 또는 None
    """
    url = "https://apply.gh.or.kr/sb/sr/sr7150/selectPbancDetailView.do"
    
    # Form Data 구성 (Payload에서 확인한 내용)
    data = {
        'searchArea': '',
        'searchCate': '',
        'searchState': '',
        'searchTitle': '',
        'previewYn': 'N',
        'pbancNo': str(pbanc_no),
        'pbancKndCd': str(pbanc_knd_cd),
        'bizTyNm': biz_ty_nm,
        'pageIndex': '1',
        'gvPgmId': 'SR7150M00'
    }
    
    try:
        response = requests.post(url, data=data, timeout=10)
        
        if response.status_code == 200:
            return response.text
        else:
            print(f"  ❌ 오류: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"  ❌ 예외: {str(e)}")
        return None

print("✅ 함수 정의 완료: get_notice_detail_by_post()")
print()

# 테스트
print("="*80)
print("🧪 POST 요청 테스트 (공고번호: 762)")
print("="*80)
print()

print("📡 POST 요청 전송 중...")
html = get_notice_detail_by_post(pbanc_no=762, pbanc_knd_cd='01', biz_ty_nm='국민임대')

if html:
    print("✅ 성공!")
    print(f"  • HTML 길이: {len(html):,} 자")
    print(f"  • 앞 200자: {html[:200]}")
    print()
    print("💡 이 HTML을 BeautifulSoup으로 파싱하면 상세 정보 추출 가능!")
else:
    print("❌ 실패")

print()
print("="*80)


🔍 POST 요청 함수 생성

✅ 함수 정의 완료: get_notice_detail_by_post()

🧪 POST 요청 테스트 (공고번호: 762)

📡 POST 요청 전송 중...
✅ 성공!
  • HTML 길이: 34,264 자
  • 앞 200자: 
















<!-- HEADER BEGIN -->



<!DOCTYPE html>
<html lang="ko">
<head>
	<meta http-equiv="Content-Type" content="text/html;charset=UTF-8">
	<meta http-equiv="X-UA-C

💡 이 HTML을 BeautifulSoup으로 파싱하면 상세 정보 추출 가능!



## 🔧 API 데이터와 POST 파라미터 매핑

API에서 제공하는 데이터를 POST 요청 파라미터로 변환해야 합니다.

### 필요한 매핑:

1. **`구분`** → **`pbancKndCd`** (공고 종류 코드)
2. 공고 유형 추정 → **`bizTyNm`** (사업 유형명)

API 데이터를 먼저 확인해서 매핑 규칙을 만들어야 합니다.


In [ ]:
# 실제 Payload 데이터 검증 (Chrome 개발자 도구에서 수집)
print("="*80)
print("🔍 실제 Payload 데이터 검증")
print("="*80)
print()

# Chrome 개발자 도구에서 수집한 실제 Payload 데이터
real_payloads = [
    {
        'title': '용인영덕 경기행복주택 입주자 추가모집 공고',
        'pbancNo': 736,
        'pbancKndCd': '01',
        'bizTyNm': '행복주택'
    },
    {
        'title': '(최초) 광주역세권 청년혁신타운 통합공공임대주택(일자리연계형 지원주택) 입주자 모집 공고',
        'pbancNo': 742,
        'pbancKndCd': '01',
        'bizTyNm': '통합공공임대'
    },
    {
        'title': '반월역 두산위브더센트럴 장기전세 입주자 모집공고',
        'pbancNo': 728,
        'pbancKndCd': '01',
        'bizTyNm': '장기전세'
    },
    {
        'title': '[정정공고]안성공도 참아름 2단지 국민임대주택 예비입주자 모집공고',
        'pbancNo': 709,
        'pbancKndCd': '02',
        'bizTyNm': '국민임대'
    },
    {
        'title': '다산 센트럴파크6단지(지금A1) 영구임대주택 입주완화 입주자 및 예비자 모집',
        'pbancNo': 676,
        'pbancKndCd': '01',
        'bizTyNm': '영구임대'
    }
]

print("📋 Chrome 개발자 도구에서 수집한 실제 Payload:")
print("─"*80)
for idx, payload in enumerate(real_payloads, 1):
    print(f"[{idx}] {payload['title'][:40]}...")
    print(f"    pbancNo: {payload['pbancNo']} | pbancKndCd: {payload['pbancKndCd']} | bizTyNm: {payload['bizTyNm']}")
print()

# API에서 동일한 공고 조회하여 비교
print("="*80)
print("🔍 API 데이터와 실제 Payload 비교")
print("="*80)
print()

# 공고번호로 API 데이터 조회
target_pbanc_nos = [736, 742, 728, 709, 676]

print("📡 API에서 해당 공고들 검색 중...")
all_api_data = []
for page in range(1, 10):  # 처음 10페이지
    result = get_gh_data(page=page, per_page=100, year='2025', debug=False)
    if result and 'data' in result:
        all_api_data.extend(result['data'])
    else:
        break

df_api = pd.DataFrame(all_api_data)

print(f"✅ API 데이터 수집 완료: 총 {len(df_api)}건\n")

if '공고번호' in df_api.columns:
    print("="*80)
    print("📊 API 데이터 vs 실제 Payload 비교")
    print("="*80)
    print()
    
    for payload in real_payloads:
        pbanc_no = payload['pbancNo']
        matches = df_api[df_api['공고번호'] == pbanc_no]
        
        if len(matches) > 0:
            api_row = matches.iloc[0]
            api_gubun = api_row.get('구분', 'N/A')
            api_name = api_row.get('공고명', 'N/A')
            
            print(f"공고번호 {pbanc_no}:")
            print(f"  ✅ API에서 발견")
            print(f"  • API 공고명: {api_name[:50]}...")
            print(f"  • API 구분: {api_gubun}")
            print(f"  • Payload pbancKndCd: {payload['pbancKndCd']}")
            print(f"  • Payload bizTyNm: {payload['bizTyNm']}")
            print(f"  • 매칭: {'✅' if str(api_gubun).zfill(2) == payload['pbancKndCd'] else '❌'}")
            print()
        else:
            print(f"공고번호 {pbanc_no}:")
            print(f"  ❌ API에서 미발견 (2025-07-31 이후 공고)")
            print(f"  • 예상 제목: {payload['title'][:50]}...")
            print(f"  • Payload pbancKndCd: {payload['pbancKndCd']}")
            print(f"  • Payload bizTyNm: {payload['bizTyNm']}")
            print()
    
    print("="*80)
    print("💡 발견 사항")
    print("="*80)
    print()
    print("✅ pbancKndCd 매핑 규칙:")
    print("  • API '구분' 필드를 2자리 문자열로 변환")
    print("  • 예: 구분 1 → '01', 구분 2 → '02'")
    print()
    print("✅ bizTyNm 추출 규칙:")
    print("  • 공고명에서 키워드 추출 (행복, 국민, 통합, 전세, 영구)")
    print("  • 실제 Payload와 대부분 일치")
    print()
else:
    print("❌ API 데이터에 '공고번호' 컬럼이 없습니다")

print("="*80)


🔍 실제 Payload 데이터 검증

📋 Chrome 개발자 도구에서 수집한 실제 Payload:
────────────────────────────────────────────────────────────────────────────────
[1] 용인영덕 경기행복주택 입주자 추가모집 공고...
    pbancNo: 736 | pbancKndCd: 01 | bizTyNm: 행복주택
[2] (최초) 광주역세권 청년혁신타운 통합공공임대주택...
    pbancNo: 742 | pbancKndCd: 01 | bizTyNm: 통합공공임대
[3] 반월역 두산위브더센트럴 장기전세 입주자 모집공고...
    pbancNo: 728 | pbancKndCd: 01 | bizTyNm: 장기전세
[4] [정정공고]안성공도 참아름 2단지 국민임대주택...
    pbancNo: 709 | pbancKndCd: 02 | bizTyNm: 국민임대
[5] 다산 센트럴파크6단지(지금A1) 영구임대주택...
    pbancNo: 676 | pbancKndCd: 01 | bizTyNm: 영구임대

🔍 API 데이터와 실제 Payload 비교

📡 API에서 해당 공고들 검색 중...
✅ API 데이터 수집 완료: 총 900건

📊 API 데이터 vs 실제 Payload 비교

공고번호 736:
  ❌ API에서 미발견 (2025-07-31 이후 공고)
  • 예상 제목: 용인영덕 경기행복주택 입주자 추가모집 공고...
  • Payload pbancKndCd: 01
  • Payload bizTyNm: 행복주택

공고번호 742:
  ❌ API에서 미발견 (2025-07-31 이후 공고)
  • 예상 제목: (최초) 광주역세권 청년혁신타운 통합공공임대주택...
  • Payload pbancKndCd: 01
  • Payload bizTyNm: 통합공공임대

공고번호 728:
  ❌ API에서 미발견 (2025-07-31 이후 공고)
  • 예상 제목: 반월역 두산위브더센트럴 장기전

In [21]:
# ⚠️ 수정된 검증: 전체 API 데이터 수집 후 최신 순 정렬
print("="*80)
print("🔍 실제 Payload 검증 (전체 API 데이터 기반)")
print("="*80)
print()
print("⚠️  중요: API는 오래된 데이터부터 반환합니다!")
print("   → 전체 데이터를 수집한 후 최신 순으로 정렬하여 확인합니다.")
print()

# Chrome 개발자 도구에서 수집한 실제 Payload 데이터
real_payloads = [
    {'title': '용인영덕 경기행복주택 입주자 추가모집 공고', 'pbancNo': 736, 'pbancKndCd': '01', 'bizTyNm': '행복주택'},
    {'title': '(최초) 광주역세권 청년혁신타운 통합공공임대주택', 'pbancNo': 742, 'pbancKndCd': '01', 'bizTyNm': '통합공공임대'},
    {'title': '반월역 두산위브더센트럴 장기전세 입주자 모집공고', 'pbancNo': 728, 'pbancKndCd': '01', 'bizTyNm': '장기전세'},
    {'title': '[정정공고]안성공도 참아름 2단지 국민임대주택', 'pbancNo': 709, 'pbancKndCd': '02', 'bizTyNm': '국민임대'},
    {'title': '다산 센트럴파크6단지(지금A1) 영구임대주택', 'pbancNo': 676, 'pbancKndCd': '01', 'bizTyNm': '영구임대'}
]

# API 전체 데이터 수집
print("📡 API 전체 데이터 수집 중...")
all_api_data = []
page = 1

while page <= 20:  # 최대 20페이지
    result = get_gh_data(page=page, per_page=100, year='2025', debug=False)
    
    if result and 'data' in result and len(result['data']) > 0:
        all_api_data.extend(result['data'])
        
        if page % 5 == 0:
            print(f"  📄 페이지 {page} 처리 중... (누적: {len(all_api_data)}건)")
        
        if len(result['data']) < 100:
            break
        page += 1
    else:
        break

print(f"✅ API 데이터 수집 완료: 총 {len(all_api_data)}건\n")

# DataFrame 변환 및 정렬
df_api = pd.DataFrame(all_api_data)

if '공고번호' in df_api.columns and '공고일자' in df_api.columns:
    # 공고일자로 정렬 (최신 순)
    df_api_sorted = df_api.sort_values('공고일자', ascending=False).reset_index(drop=True)
    
    print("="*80)
    print("📊 API 최신 30개 데이터 (공고일자 기준)")
    print("="*80)
    print()
    
    # 최신 30개 출력
    for idx, row in df_api_sorted.head(30).iterrows():
        pbanc_no = row.get('공고번호', 'N/A')
        pbanc_date = str(row.get('공고일자', 'N/A'))
        pbanc_name = str(row.get('공고명', 'N/A'))[:45]
        gubun = row.get('구분', 'N/A')
        
        # 날짜 포맷팅
        if len(pbanc_date) == 8:
            pbanc_date_fmt = f"{pbanc_date[:4]}-{pbanc_date[4:6]}-{pbanc_date[6:]}"
        else:
            pbanc_date_fmt = pbanc_date
        
        # 검증 대상 공고 표시
        marker = " ⭐" if pbanc_no in [736, 742, 728, 709, 676] else ""
        
        print(f"[{idx+1:2d}] 번호:{pbanc_no:4} | 일자:{pbanc_date_fmt} | 구분:{gubun:2} | {pbanc_name}...{marker}")
    
    print()
    print("="*80)
    print("🔍 Payload 검증 대상 5개 공고 상세 확인")
    print("="*80)
    print()
    
    for payload in real_payloads:
        pbanc_no = payload['pbancNo']
        matches = df_api[df_api['공고번호'] == pbanc_no]
        
        if len(matches) > 0:
            api_row = matches.iloc[0]
            
            # API 데이터
            api_gubun = api_row.get('구분', 'N/A')
            api_name = api_row.get('공고명', 'N/A')
            api_date = str(api_row.get('공고일자', 'N/A'))
            
            if len(api_date) == 8:
                api_date_fmt = f"{api_date[:4]}-{api_date[4:6]}-{api_date[6:]}"
            else:
                api_date_fmt = api_date
            
            # pbancKndCd 변환
            api_knd_cd = str(api_gubun).zfill(2)
            knd_cd_match = api_knd_cd == payload['pbancKndCd']
            
            print(f"━━━ 공고번호: {pbanc_no} ━━━")
            print(f"  ✅ API 발견")
            print(f"  📋 API:     {api_name[:50]}... | 구분:{api_gubun} → '{api_knd_cd}'")
            print(f"  🌐 Payload: {payload['title'][:50]}... | '{payload['pbancKndCd']}'")
            print(f"  🔍 매칭:    {api_knd_cd} vs {payload['pbancKndCd']} → {'✅일치' if knd_cd_match else '❌불일치'}")
            print()
        else:
            print(f"━━━ 공고번호: {pbanc_no} ━━━")
            print(f"  ❌ API 미발견 (2025-07-31 이후)")
            print(f"  🌐 Payload: {payload['title'][:50]}...")
            print()
    
    print("="*80)
    print("💡 검증 결과")
    print("="*80)
    found = sum(1 for p in real_payloads if p['pbancNo'] in df_api['공고번호'].values)
    print(f"  검증 대상: {len(real_payloads)}개 | API 발견: {found}개 | 미발견: {len(real_payloads)-found}개")
    if found == len(real_payloads):
        print("  ✅ 모두 발견! pbancKndCd 매핑: API '구분' → 2자리 문자열 ('01', '02', ...)")
    print()
else:
    print("❌ '공고번호' 또는 '공고일자' 컬럼 없음")

print("="*80)


🔍 실제 Payload 검증 (전체 API 데이터 기반)

⚠️  중요: API는 오래된 데이터부터 반환합니다!
   → 전체 데이터를 수집한 후 최신 순으로 정렬하여 확인합니다.

📡 API 전체 데이터 수집 중...
  📄 페이지 5 처리 중... (누적: 500건)
  📄 페이지 10 처리 중... (누적: 990건)
✅ API 데이터 수집 완료: 총 990건

📊 API 최신 30개 데이터 (공고일자 기준)

[ 1] 번호: 625 | 일자:2028-11-04 | 구분:662 | 예비자 우선공급 테스트용...
[ 2] 번호: 742 | 일자:2025-07-31 | 구분:990 | (최초) 광주역세권 청년혁신타운 통합공공임대주택(일자리연계형 지원주택) 입주자 모... ⭐
[ 3] 번호: 741 | 일자:2025-07-24 | 구분:989 | 안성청사복합 통합공공임대주택 단지 내 공공임대상가 임차인 2차 모집 공고...
[ 4] 번호: 740 | 일자:2025-07-18 | 구분:988 | (선착순수의계약) 안성청사복합 통합공공임대주택 단지 내 일반형 임대상가 임차인 모...
[ 5] 번호: 739 | 일자:2025-07-18 | 구분:987 | (선착순수의계약) 용인영덕 경기행복주택 단지 내 일반형 임대상가 임차인 모집 공고...
[ 6] 번호: 738 | 일자:2025-07-18 | 구분:986 | (선착순수의계약) 수원영통 경기행복주택 단지 내 일반형 임대상가 임차인 모집 공고...
[ 7] 번호: 737 | 일자:2025-07-18 | 구분:985 | (선착순수의계약)	양평공흥 경기행복주택 단지 내 일반형 임대상가 임차인 모집 공고...
[ 8] 번호: 736 | 일자:2025-07-10 | 구분:984 | 용인영덕 경기행복주택 입주자 추가모집 공고... ⭐
[ 9] 번호: 732 | 일자:2025-06-26 | 구분:980 | 안성청사복합 통합공공임대주택 단지 내 공공임대상가 임차인 모집 공고...
[10] 번호: 733 | 일자:2025-06-2

## 🎯 **최종 전략: API 제공 공고 vs API 미제공 최신 공고**

### 📊 **현황 분석**

#### ✅ **API 제공 공고 (2025-07-31까지)**
- API로 조회 가능
- POST 요청 정보 생성 가능
- 예: 공고번호 736, 742, 728, 709, 676 등

#### ❌ **API 미제공 최신 공고 (2025-08-01 이후)**
- API로 조회 불가 (최신 10개)
- 웹사이트에만 존재
- 공고번호: 762, 761, 757, 755, 754, 753, 743, 747, 745, 744

---

### 🔧 **해결 방안**

#### **방안 1: API 제공 공고만 CSV 생성 (권장)**
- **장점**: 정확한 데이터, 안정적
- **단점**: 최신 10개 공고 누락
- **용도**: 과거 공고 일괄 크롤링

#### **방안 2: 최신 공고는 Selenium으로 수동 크롤링**
- **장점**: 모든 공고 커버
- **단점**: 추가 작업 필요
- **방법**: 
  1. 목록 페이지 접속
  2. 1페이지 공고 순회하며 크롤링
  3. 공고번호 확인 후 데이터 추출

---

### 📁 **CSV 구조 (최종)**

```
공고번호 | 공고명 | 공고일자 | ... | data_source | post_url | pbancKndCd | bizTyNm | list_url
─────────────────────────────────────────────────────────────────────────────────────────────
736 | 용인영덕... | 20250620 | ... | API | https://... | 01 | 행복주택 | https://...
742 | 광주역세권... | 20250620 | ... | API | https://... | 01 | 통합공공 | https://...
```

**추가 컬럼:**
- `data_source`: 'API' (API 제공 공고)
- `post_url`: POST 요청 URL
- `pbancKndCd`: 공고 종류 코드
- `bizTyNm`: 사업 유형명
- `list_url`: 목록 페이지 URL

---


In [ ]:
# POST 요청 정보를 포함한 데이터 수집 함수 (최종 버전)
def collect_all_notices_with_post_info(min_year='2024', exclude_test=True):
    """
    전체 공고 데이터 수집 및 POST 요청 정보 추가
    
    Parameters:
        min_year (str): 최소 연도 (기본값: '2024')
        exclude_test (bool): 테스트 데이터 제외 여부 (기본값: True)
    
    Returns:
        DataFrame: POST 요청 정보가 포함된 공고 데이터
    """
    print("="*80)
    print(f"📡 {min_year}년 이후 전체 공고 데이터 수집 (POST 정보 포함)")
    print("="*80)
    print()
    
    # 1. API 전체 데이터 수집
    print("📊 1단계: API 데이터 수집 중...")
    all_notices = []
    page = 1
    
    while True:
        result = get_gh_data(page=page, per_page=100, year='2025', debug=False)
        
        if result and 'data' in result and len(result['data']) > 0:
            all_notices.extend(result['data'])
            
            if page % 5 == 0:
                print(f"  📄 페이지 {page} 처리 중... (누적: {len(all_notices)}건)")
            
            if len(result['data']) < 100:
                break
            
            page += 1
            
            if page > 20:  # 최대 20페이지
                break
        else:
            break
    
    print(f"  ✅ 총 {len(all_notices)}건 수집 완료\n")
    
    if not all_notices:
        print("❌ 데이터 수집 실패")
        return None
    
    # 2. DataFrame으로 변환
    print("📊 2단계: DataFrame 변환 및 전처리...")
    df_all = pd.DataFrame(all_notices)
    print(f"  • 초기 데이터: {len(df_all)}건")
    
    # 3. 필터링 (기존과 동일)
    if '공고일자' not in df_all.columns:
        print("  ❌ '공고일자' 컬럼이 없습니다")
        return None
    
    df_filtered = df_all[df_all['공고일자'].notna()].copy()
    print(f"  • 공고일자 있음: {len(df_filtered)}건")
    
    df_filtered['공고일자_str'] = df_filtered['공고일자'].astype(str)
    
    min_date = f"{min_year}0101"
    df_filtered = df_filtered[df_filtered['공고일자_str'] >= min_date].copy()
    print(f"  • {min_year}년 이후: {len(df_filtered)}건")
    
    if exclude_test:
        df_filtered = df_filtered[
            ~df_filtered['공고명'].str.contains('테스트|test|Test', case=False, na=False)
        ].copy()
        print(f"  • 테스트 제외: {len(df_filtered)}건")
    
    # 4. 중복 제거
    print(f"\n📊 3단계: 중복 제거...")
    initial_count = len(df_filtered)
    df_filtered = df_filtered.drop_duplicates(
        subset=['공고번호', '공고일자', '구분'],
        keep='first'
    ).copy()
    removed_duplicates = initial_count - len(df_filtered)
    print(f"  • 중복 제거: {removed_duplicates}건")
    print(f"  • 최종 데이터: {len(df_filtered)}건")
    
    # 5. 날짜순 정렬
    print(f"\n📊 4단계: 날짜순 정렬...")
    df_sorted = df_filtered.sort_values('공고일자', ascending=False).reset_index(drop=True)
    
    if len(df_sorted) > 0:
        latest = df_sorted.iloc[0]['공고일자_str']
        oldest = df_sorted.iloc[-1]['공고일자_str']
        latest_formatted = f"{latest[:4]}-{latest[4:6]}-{latest[6:]}"
        oldest_formatted = f"{oldest[:4]}-{oldest[4:6]}-{oldest[6:]}"
        print(f"  • 날짜 범위: {oldest_formatted} ~ {latest_formatted}")
    
    # 6. POST 요청 정보 추가 (핵심!)
    print(f"\n📊 5단계: POST 요청 정보 추가...")
    
    # 공고명에서 사업 유형 추정 (실제 Payload 데이터 기반)
    def extract_biz_type(notice_name):
        """
        공고명에서 사업 유형 추정
        Chrome 개발자 도구에서 수집한 실제 Payload 데이터 기반
        """
        if pd.isna(notice_name):
            return '기타'
        
        notice_name_str = str(notice_name)
        
        # 우선순위: 행복 > 영구 > 국민 > 통합/공공 > 전세 > 상가
        if '행복' in notice_name_str:
            return '행복주택'
        elif '영구' in notice_name_str:
            return '영구임대'
        elif '국민' in notice_name_str:
            return '국민임대'
        elif '통합' in notice_name_str or '공공임대' in notice_name_str:
            return '통합공공임대'
        elif '전세' in notice_name_str:
            return '장기전세'
        elif '상가' in notice_name_str:
            return '임대상가'
        else:
            return '기타'
    
    df_sorted['bizTyNm'] = df_sorted['공고명'].apply(extract_biz_type)
    
    # 데이터 소스 (API 제공 공고)
    df_sorted['data_source'] = 'API'
    
    # POST 요청 URL
    df_sorted['post_url'] = 'https://apply.gh.or.kr/sb/sr/sr7150/selectPbancDetailView.do'
    
    # POST 파라미터 (pbancNo, pbancKndCd)
    # 구분 → pbancKndCd로 매핑 (문자열로 변환, 2자리)
    df_sorted['pbancKndCd'] = df_sorted['구분'].astype(str).str.zfill(2)  # 01, 02, ... 형식
    
    # 목록 페이지 URL (Selenium 크롤링용)
    df_sorted['list_url'] = GH_LIST_URL
    
    print(f"  ✅ POST 정보 추가 완료")
    print(f"     • data_source: API")
    print(f"     • post_url: {df_sorted.iloc[0]['post_url']}")
    print(f"     • bizTyNm 분포: {df_sorted['bizTyNm'].value_counts().to_dict()}")
    print(f"     • pbancKndCd 분포: {df_sorted['pbancKndCd'].value_counts().to_dict()}")
    
    print()
    print("="*80)
    print("✅ 데이터 수집 및 POST 정보 추가 완료!")
    print("="*80)
    print()
    print("📊 최종 통계:")
    print("─"*80)
    print(f"  • 전체 데이터: {len(df_sorted)}건")
    print(f"  • 컬럼 수: {len(df_sorted.columns)}개")
    print(f"  • POST 정보: 포함 ✅")
    print()
    
    return df_sorted

print("✅ 함수 정의 완료: collect_all_notices_with_post_info()")


## 🚀 **최종 실행: 2024-2025 공고 데이터 + POST 정보 CSV 저장**

이제 POST 요청 정보가 포함된 CSV를 생성합니다!


In [ ]:
# 🚀 최종 실행: API 제공 공고 CSV 생성 (POST 정보 포함)
print("="*80)
print("🚀 GH 공고 데이터 수집 및 CSV 생성 (POST 정보 포함)")
print("="*80)
print()
print("📊 대상: 2024년 ~ 2025-07-31 (API 제공 공고)")
print("📁 파일명: GH_notices_2024_2025_with_POST.csv")
print()

# Step 1: POST 정보 포함 데이터 수집
print("="*80)
print("📡 Step 1: API 데이터 수집")
print("="*80)
print()

df_notices_post = collect_all_notices_with_post_info(min_year='2024', exclude_test=True)

if df_notices_post is not None and len(df_notices_post) > 0:
    print("\n" + "="*80)
    print("📊 Step 2: 데이터 미리보기 (최신 5개)")
    print("="*80)
    print()
    
    for idx, row in df_notices_post.head(5).iterrows():
        print(f"[{idx+1}] ─────────────────────────────────────────────────────")
        
        notice_name = str(row.get('공고명', 'N/A'))[:50]
        notice_date = str(row.get('공고일자', 'N/A'))
        if len(notice_date) == 8:
            notice_date = f"{notice_date[:4]}-{notice_date[4:6]}-{notice_date[6:]}"
        
        print(f"     공고: {notice_name}{'...' if len(str(row.get('공고명', ''))) > 50 else ''}")
        print(f"     일자: {notice_date}")
        print(f"     번호: {row.get('공고번호', 'N/A')}")
        print(f"     유형: {row.get('bizTyNm', 'N/A')} (코드: {row.get('pbancKndCd', 'N/A')})")
        print(f"     소스: {row.get('data_source', 'N/A')}")
        print()
    
    # Step 2: CSV 저장
    print("="*80)
    print("💾 Step 3: CSV 파일 저장")
    print("="*80)
    print()
    
    csv_path = save_notices_to_csv(
        df_notices_post,
        filename='GH_notices_2024_2025_with_POST',
        include_all_columns=True
    )
    
    if csv_path:
        print("\n" + "="*80)
        print("🎉 완료!")
        print("="*80)
        print()
        print("📁 생성된 파일:")
        print(f"  • {csv_path}")
        print(f"  • 데이터 건수: {len(df_notices_post)}건")
        print(f"  • 컬럼 수: {len(df_notices_post.columns)}개")
        print()
        print("📋 CSV에 포함된 POST 정보:")
        print("─"*80)
        print("  • data_source: 데이터 소스 ('API')")
        print("  • post_url: POST 요청 URL")
        print("  • pbancKndCd: 공고 종류 코드 (01, 02, ...)")
        print("  • bizTyNm: 사업 유형명 (행복주택, 국민임대, ...)")
        print("  • list_url: 목록 페이지 URL (Selenium용)")
        print()
        print("🔧 크롤링 예시 (Python requests):")
        print("─"*80)
        print("""
import requests, pandas as pd

df = pd.read_csv('output/GH_notices_2024_2025_with_POST.csv')

# 첫 번째 공고 크롤링 테스트
row = df.iloc[0]
data = {
    'searchArea': '', 'searchCate': '', 'searchState': '', 'searchTitle': '',
    'previewYn': 'N', 'pageIndex': '1', 'gvPgmId': 'SR7150M00',
    'pbancNo': str(row['공고번호']),
    'pbancKndCd': str(row['pbancKndCd']),
    'bizTyNm': row['bizTyNm']
}

response = requests.post(row['post_url'], data=data)
# BeautifulSoup으로 파싱...
        """)
        print()
        print("💾 변수:")
        print(f"  • df_notices_post: {df_notices_post.shape}")
        print()
        print("⚠️  Note: Cell 17~22, Cell 41은 이전 시도 셀입니다. 삭제 가능합니다.")
        print()
else:
    print("\n❌ 데이터 수집 실패")

print("="*80)


---

## 🎉 **프로젝트 완료 요약**

### ✅ **완료된 작업**

1. **Chrome 개발자 도구로 Payload 확인** ✅
   - POST 요청 Form Data 분석
   - pbancNo, pbancKndCd, bizTyNm 패턴 파악

2. **API 데이터와 실제 Payload 비교** ✅
   - API '구분' → pbancKndCd 매핑 확인
   - 공고명 → bizTyNm 추출 로직 검증

3. **데이터 수집 함수 개발** ✅
   - `collect_all_notices_with_post_info()`: POST 정보 포함 데이터 수집
   - 2024년~2025-07-31 (API 제공 공고) 필터링
   - 중복 제거 및 테스트 공고 제외

4. **CSV 생성 (POST 정보 포함)** ✅
   - data_source, post_url, pbancKndCd, bizTyNm, list_url 추가
   - 팀원 크롤링용 완전한 정보 제공

---

### 📊 **API 제한사항 및 대응**

| 구분 | API 제공 공고 | API 미제공 최신 공고 |
|------|--------------|---------------------|
| 기간 | 2024-01-01 ~ 2025-07-31 | 2025-08-01 ~ 현재 |
| 개수 | 약 700+건 | 약 10건 |
| 크롤링 방법 | **Python requests** (POST) | **Selenium** (목록 페이지 순회) |
| CSV 포함 | ✅ (POST 정보 포함) | ❌ (별도 크롤링 필요) |

---

### 📁 **생성된 CSV 구조**

```csv
공고번호,공고명,공고일자,...,data_source,post_url,pbancKndCd,bizTyNm,list_url
736,용인영덕...,20250620,...,API,https://...,01,행복주택,https://...
742,광주역세권...,20250620,...,API,https://...,01,통합공공임대,https://...
```

**핵심 컬럼:**
- `data_source`: 'API' (데이터 출처)
- `post_url`: POST 요청 URL
- `pbancKndCd`: 공고 종류 코드 (01, 02, ...)
- `bizTyNm`: 사업 유형명 (행복주택, 국민임대, 통합공공임대, 장기전세, 영구임대)
- `list_url`: 목록 페이지 URL (Selenium용)

---

### 🚀 **실행 방법**

```python
# 필수 셀 실행 순서:
Cell 2  → API 설정 (API_KEY, ENDPOINTS)
Cell 4  → get_gh_data 함수
Cell 16 → POST URL 정보
Cell 27 → Payload 검증 (선택)
Cell 29 → collect_all_notices_with_post_info 함수
Cell 31 → 🎯 최종 실행 (CSV 생성)
```

---

### 🔧 **크롤링 가이드**

#### **API 제공 공고 (CSV 활용)**

```python
import requests, pandas as pd
from bs4 import BeautifulSoup

df = pd.read_csv('output/GH_notices_2024_2025_with_POST.csv')

for idx, row in df.iterrows():
    data = {
        'searchArea': '', 'searchCate': '', 'searchState': '', 'searchTitle': '',
        'previewYn': 'N', 'pageIndex': '1', 'gvPgmId': 'SR7150M00',
        'pbancNo': str(row['공고번호']),
        'pbancKndCd': str(row['pbancKndCd']),
        'bizTyNm': row['bizTyNm']
    }
    
    response = requests.post(row['post_url'], data=data)
    soup = BeautifulSoup(response.text, 'html.parser')
    # 데이터 추출...
```

#### **API 미제공 최신 공고 (Selenium 필수)**

```python
from selenium import webdriver

driver = webdriver.Chrome()
driver.get('https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseList.do')

# 1페이지 공고 순회
# 공고번호 확인 → 클릭 → 크롤링
```

---

### 📝 **다음 단계**

1. **API 제공 공고 크롤링** (우선순위 1)
   - CSV 파일 확인
   - 샘플 1개 테스트
   - 전체 크롤링 (requests 권장)

2. **API 미제공 최신 공고 크롤링** (우선순위 2)
   - Selenium으로 목록 페이지 접근
   - 1~2페이지 순회
   - 공고번호: 762, 761, 757, 755, 754, 753, 743, 747, 745, 744

3. **데이터 통합 및 정제**
   - API 제공 + 최신 공고 병합
   - 중복 제거
   - 데이터베이스 저장

---

### ⚠️ **정리 권장**

- **Cell 17~22**: 이전 URL 찾기 시도 셀 → 삭제 가능
- **Cell 41**: 구버전 실행 셀 → 삭제 가능

---


In [12]:
# 목록 페이지에서 실제 링크 패턴 추출
print("="*80)
print("🔍 목록 페이지 HTML 분석")
print("="*80)
print()

try:
    from bs4 import BeautifulSoup
    
    print("📡 목록 페이지 접속 중...")
    response = requests.get(GH_LIST_URL, timeout=10)
    
    if response.status_code == 200:
        print("✅ 페이지 로드 성공!\n")
        
        # HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')
        
        print("="*80)
        print("🔍 공고 링크 분석")
        print("="*80)
        print()
        
        # 방법 1: <a> 태그에서 onclick 이벤트 찾기
        print("📋 방법 1: onClick 이벤트 분석")
        print("─"*80)
        
        # 공고 목록의 링크 찾기 (일반적으로 테이블 또는 리스트 형태)
        links = soup.find_all('a', href=True)
        
        onclick_patterns = []
        for link in links[:20]:  # 상위 20개만 확인
            onclick = link.get('onclick', '')
            if onclick and ('detail' in onclick.lower() or 'view' in onclick.lower()):
                onclick_patterns.append(onclick)
        
        if onclick_patterns:
            print("✅ onClick 이벤트 발견:")
            print()
            for i, pattern in enumerate(onclick_patterns[:5], 1):
                print(f"  [{i}] {pattern[:100]}")
            print()
        else:
            print("  ⚠️  onClick 패턴을 찾지 못했습니다")
            print()
        
        # 방법 2: href 속성 분석
        print("📋 방법 2: href 속성 분석")
        print("─"*80)
        
        detail_links = []
        for link in links:
            href = link.get('href', '')
            if 'detail' in href.lower() or 'view' in href.lower():
                detail_links.append(href)
        
        if detail_links:
            print("✅ 상세 페이지 링크 발견:")
            print()
            for i, href in enumerate(detail_links[:5], 1):
                print(f"  [{i}] {href}")
            print()
        else:
            print("  ⚠️  href 패턴을 찾지 못했습니다")
            print()
        
        # 방법 3: JavaScript 함수 분석
        print("📋 방법 3: JavaScript 함수 찾기")
        print("─"*80)
        
        scripts = soup.find_all('script')
        js_functions = []
        
        for script in scripts:
            if script.string:
                script_text = script.string
                # detail, view 관련 함수 찾기
                if 'function' in script_text and ('detail' in script_text.lower() or 'view' in script_text.lower()):
                    # 함수 시그니처 추출
                    lines = script_text.split('\n')
                    for line in lines:
                        if 'function' in line and ('detail' in line.lower() or 'view' in line.lower()):
                            js_functions.append(line.strip())
        
        if js_functions:
            print("✅ JavaScript 함수 발견:")
            print()
            for i, func in enumerate(js_functions[:5], 1):
                print(f"  [{i}] {func[:100]}")
            print()
        else:
            print("  ⚠️  JavaScript 함수를 찾지 못했습니다")
            print()
        
        # 방법 4: Form 분석
        print("📋 방법 4: Form 요소 분석")
        print("─"*80)
        
        forms = soup.find_all('form')
        if forms:
            print(f"✅ {len(forms)}개의 Form 발견:")
            print()
            for i, form in enumerate(forms[:3], 1):
                action = form.get('action', 'N/A')
                method = form.get('method', 'N/A')
                form_id = form.get('id', 'N/A')
                print(f"  [{i}] Form ID: {form_id}")
                print(f"      Action: {action}")
                print(f"      Method: {method}")
                print()
        else:
            print("  ⚠️  Form을 찾지 못했습니다")
            print()
        
        print("="*80)
        print("💡 분석 결과 요약")
        print("="*80)
        print()
        print("위의 결과를 바탕으로:")
        print("  1. onClick 이벤트의 함수명과 파라미터 확인")
        print("  2. href 또는 Form action의 URL 패턴 확인")
        print("  3. JavaScript 함수의 동작 방식 분석")
        print()
        print("💡 다음 단계:")
        print("  • 발견된 패턴을 바탕으로 URL 생성 함수 수정")
        print("  • 또는 목록 페이지에서 직접 링크 추출하는 방식 사용")
        print()
        
    else:
        print(f"❌ 페이지 로드 실패: {response.status_code}")

except ImportError:
    print("❌ BeautifulSoup4가 설치되지 않았습니다")
    print()
    print("설치 방법:")
    print("  pip install beautifulsoup4")
    print()

except Exception as e:
    print(f"❌ 오류 발생: {str(e)}")

print("="*80)


🔍 목록 페이지 HTML 분석

📡 목록 페이지 접속 중...
✅ 페이지 로드 성공!

🔍 공고 링크 분석

📋 방법 1: onClick 이벤트 분석
────────────────────────────────────────────────────────────────────────────────
  ⚠️  onClick 패턴을 찾지 못했습니다

📋 방법 2: href 속성 분석
────────────────────────────────────────────────────────────────────────────────
✅ 상세 페이지 링크 발견:

  [1] javascript:$net.href('/cs/sr/sr7860/selectIssuCrtfView.do');
  [2] javascript:$net.href('/cs/sr/sr7870/selectSplyCenterInfoView.do');
  [3] /cs/sr/sr7870/selectSplyCenterInfoView.do
  [4] javascript:$net.href('/co/coa/selectMainView.do');

📋 방법 3: JavaScript 함수 찾기
────────────────────────────────────────────────────────────────────────────────
  ⚠️  JavaScript 함수를 찾지 못했습니다

📋 방법 4: Form 요소 분석
────────────────────────────────────────────────────────────────────────────────
✅ 2개의 Form 발견:

  [1] Form ID: bbsSearchFrm
      Action: N/A
      Method: N/A

  [2] Form ID: competitionFrm
      Action: N/A
      Method: N/A

💡 분석 결과 요약

위의 결과를 바탕으로:
  1. onClick 이벤트의 함수명과 파라미터 확인
  2

## 📋 대안: 목록 페이지 기준 URL 사용

실제 공고 상세 URL을 파라미터로 만들 수 없다면, **목록 페이지 URL**을 제공하는 것이 가장 안전합니다.


In [13]:
# 대안: 목록 페이지 URL 또는 검색 가능한 URL 제공
print("="*80)
print("💡 대안: 실용적인 URL 제공 방식")
print("="*80)
print()

# GH 홈페이지 기본 URL
GH_BASE_URL = "https://apply.gh.or.kr"
GH_LIST_URL = f"{GH_BASE_URL}/sb/sr/sr7150/selectPbancRentHouseList.do"

print("📋 3가지 URL 제공 방식:")
print("="*80)
print()

# 방법 1: 목록 페이지 URL (가장 안전)
def generate_list_page_url():
    """목록 페이지 URL 반환"""
    return GH_LIST_URL

print("방법 1: 목록 페이지 URL (권장)")
print("─"*80)
print(f"  URL: {generate_list_page_url()}")
print()
print("  장점:")
print("    ✅ 항상 정확하게 작동")
print("    ✅ 공고 검색 가능")
print("    ✅ 사용자가 직접 공고 찾기")
print()
print("  단점:")
print("    ❌ 특정 공고로 바로 이동 불가")
print("    ❌ 수동으로 검색 필요")
print()

# 방법 2: 목록 페이지 + 공고번호 (검색 힌트)
def generate_search_hint_url(notice_number):
    """목록 페이지 URL + 공고번호 힌트"""
    return f"{GH_LIST_URL}#notice-{notice_number}"

print("방법 2: 목록 페이지 + 공고번호 힌트")
print("─"*80)
sample_url = generate_search_hint_url(742)
print(f"  URL: {sample_url}")
print()
print("  장점:")
print("    ✅ 공고번호 정보 포함")
print("    ✅ 목록 페이지 접근 가능")
print()
print("  단점:")
print("    ❌ 해시(#)는 서버에 전달되지 않음")
print("    ❌ 여전히 수동 검색 필요")
print()

# 방법 3: 목록 페이지 + 검색 파라미터 (시도)
def generate_search_param_url(notice_number=None, notice_name=None):
    """
    목록 페이지 URL + 검색 파라미터
    (실제 동작 여부는 테스트 필요)
    """
    url = GH_LIST_URL
    params = []
    
    if notice_number:
        params.append(f"searchPbancNo={notice_number}")
    if notice_name:
        # URL 인코딩 필요
        import urllib.parse
        encoded_name = urllib.parse.quote(notice_name)
        params.append(f"searchPbancNm={encoded_name}")
    
    if params:
        url += "?" + "&".join(params)
    
    return url

print("방법 3: 목록 페이지 + 검색 파라미터 (시도)")
print("─"*80)
sample_url_3 = generate_search_param_url(notice_number=742)
print(f"  URL: {sample_url_3}")
print()
print("  장점:")
print("    ✅ 검색 자동화 시도")
print("    ✅ 공고번호로 필터링 가능성")
print()
print("  단점:")
print("    ❌ 실제 파라미터 이름이 다를 수 있음")
print("    ❌ 동작 보장 불가")
print()

print("="*80)
print("🎯 최종 권장사항")
print("="*80)
print()
print("📋 CSV 파일의 URL 컬럼에 제공할 내용:")
print()
print("1️⃣ 공고 상세 페이지에 직접 접근할 수 없는 경우:")
print("   → 목록 페이지 URL + 공고번호를 별도 컬럼으로 제공")
print()
print("   예시:")
print("   ┌─────────────────────────────────────────────────────┐")
print("   │ 공고번호 │ 공고명        │ list_url     │ notice_no │")
print("   ├─────────────────────────────────────────────────────┤")
print("   │ 742      │ 광주역세권... │ https://...  │ 742       │")
print("   └─────────────────────────────────────────────────────┘")
print()

print("2️⃣ 팀원들은 Selenium으로 크롤링:")
print("   → 목록 페이지 접속")
print("   → 공고번호로 검색 또는 찾기")
print("   → 해당 공고 클릭")
print("   → 상세 페이지 크롤링")
print()

print("3️⃣ 개발자 도구 확인 후 실제 패턴 발견 시:")
print("   → generate_notice_url() 함수 수정")
print("   → URL 컬럼 업데이트")
print()

print("="*80)

# 함수 정의
def generate_notice_url_v2(notice_number, use_list_page=True):
    """
    공고번호로 URL 생성 (버전 2)
    
    Parameters:
        notice_number: 공고번호
        use_list_page (bool): 목록 페이지 사용 여부 (기본값: True)
    
    Returns:
        str: 공고 URL
    """
    if use_list_page:
        # 안전한 방법: 목록 페이지 URL 반환
        return GH_LIST_URL
    else:
        # 시도: 검색 파라미터 추가 (동작 보장 불가)
        return f"{GH_LIST_URL}?searchPbancNo={notice_number}"

print("✅ 함수 정의 완료:")
print("  • generate_list_page_url()")
print("  • generate_search_hint_url(notice_number)")
print("  • generate_search_param_url(notice_number, notice_name)")
print("  • generate_notice_url_v2(notice_number, use_list_page=True)")


💡 대안: 실용적인 URL 제공 방식

📋 3가지 URL 제공 방식:

방법 1: 목록 페이지 URL (권장)
────────────────────────────────────────────────────────────────────────────────
  URL: https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseList.do

  장점:
    ✅ 항상 정확하게 작동
    ✅ 공고 검색 가능
    ✅ 사용자가 직접 공고 찾기

  단점:
    ❌ 특정 공고로 바로 이동 불가
    ❌ 수동으로 검색 필요

방법 2: 목록 페이지 + 공고번호 힌트
────────────────────────────────────────────────────────────────────────────────
  URL: https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseList.do#notice-742

  장점:
    ✅ 공고번호 정보 포함
    ✅ 목록 페이지 접근 가능

  단점:
    ❌ 해시(#)는 서버에 전달되지 않음
    ❌ 여전히 수동 검색 필요

방법 3: 목록 페이지 + 검색 파라미터 (시도)
────────────────────────────────────────────────────────────────────────────────
  URL: https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseList.do?searchPbancNo=742

  장점:
    ✅ 검색 자동화 시도
    ✅ 공고번호로 필터링 가능성

  단점:
    ❌ 실제 파라미터 이름이 다를 수 있음
    ❌ 동작 보장 불가

🎯 최종 권장사항

📋 CSV 파일의 URL 컬럼에 제공할 내용:

1️⃣ 공고 상세 페이지에 직접 접근할 수 없는 경우:
   → 목록 페이지 URL + 공고번호를 별도 컬럼으로 제공

   예

## 📋 Step 2: 전체 공고 데이터 수집 함수

2024-2025년 전체 공고를 수집하고, 중복 제거 및 테스트 공고를 필터링합니다.


In [ ]:
def collect_all_notices_with_url(min_year='2024', exclude_test=True):
    """
    전체 공고 데이터 수집 및 URL 추가
    
    Parameters:
        min_year (str): 최소 연도 (기본값: '2024')
        exclude_test (bool): 테스트 데이터 제외 여부 (기본값: True)
    
    Returns:
        DataFrame: URL이 포함된 공고 데이터
    """
    print("="*80)
    print(f"📡 {min_year}년 이후 전체 공고 데이터 수집")
    print("="*80)
    print()
    
    # 1. API 전체 데이터 수집
    print("📊 1단계: API 데이터 수집 중...")
    all_notices = []
    page = 1
    
    while True:
        result = get_gh_data(page=page, per_page=100, year='2025', debug=False)
        
        if result and 'data' in result and len(result['data']) > 0:
            all_notices.extend(result['data'])
            
            # 진행 상황 출력 (5페이지마다)
            if page % 5 == 0:
                print(f"  📄 페이지 {page} 처리 중... (누적: {len(all_notices)}건)")
            
            if len(result['data']) < 100:
                break
            
            page += 1
            
            # 안전장치: 최대 20페이지 (2000건)
            if page > 20:
                break
        else:
            break
    
    print(f"  ✅ 총 {len(all_notices)}건 수집 완료\n")
    
    if not all_notices:
        print("❌ 데이터 수집 실패")
        return None
    
    # 2. DataFrame으로 변환
    print("📊 2단계: DataFrame 변환 및 전처리...")
    df_all = pd.DataFrame(all_notices)
    print(f"  • 초기 데이터: {len(df_all)}건")
    
    # 3. 필터링
    # 3-1. 공고일자가 없는 데이터 제거
    if '공고일자' not in df_all.columns:
        print("  ❌ '공고일자' 컬럼이 없습니다")
        return None
    
    df_filtered = df_all[df_all['공고일자'].notna()].copy()
    print(f"  • 공고일자 있음: {len(df_filtered)}건")
    
    # 3-2. 날짜 문자열로 변환
    df_filtered['공고일자_str'] = df_filtered['공고일자'].astype(str)
    
    # 3-3. 지정된 연도 이후만 필터
    min_date = f"{min_year}0101"
    df_filtered = df_filtered[df_filtered['공고일자_str'] >= min_date].copy()
    print(f"  • {min_year}년 이후: {len(df_filtered)}건")
    
    # 3-4. 테스트 데이터 제외
    if exclude_test:
        df_filtered = df_filtered[
            ~df_filtered['공고명'].str.contains('테스트|test|Test', case=False, na=False)
        ].copy()
        print(f"  • 테스트 제외: {len(df_filtered)}건")
    
    # 4. 중복 제거
    print(f"\n📊 3단계: 중복 제거...")
    
    # 공고번호 + 공고일자 + 구분으로 중복 판단
    initial_count = len(df_filtered)
    df_filtered = df_filtered.drop_duplicates(
        subset=['공고번호', '공고일자', '구분'],
        keep='first'
    ).copy()
    removed_duplicates = initial_count - len(df_filtered)
    print(f"  • 중복 제거: {removed_duplicates}건")
    print(f"  • 최종 데이터: {len(df_filtered)}건")
    
    # 5. 날짜순 정렬 (최신순)
    print(f"\n📊 4단계: 날짜순 정렬...")
    df_sorted = df_filtered.sort_values('공고일자', ascending=False).reset_index(drop=True)
    
    # 날짜 범위 확인
    if len(df_sorted) > 0:
        latest = df_sorted.iloc[0]['공고일자_str']
        oldest = df_sorted.iloc[-1]['공고일자_str']
        latest_formatted = f"{latest[:4]}-{latest[4:6]}-{latest[6:]}"
        oldest_formatted = f"{oldest[:4]}-{oldest[4:6]}-{oldest[6:]}"
        print(f"  • 날짜 범위: {oldest_formatted} ~ {latest_formatted}")
    
    # 6. URL 컬럼 추가
    print(f"\n📊 5단계: 공고 URL 추가...")
    df_sorted['url'] = df_sorted['공고번호'].apply(generate_notice_url)
    print(f"  ✅ URL 컬럼 추가 완료")
    
    print()
    print("="*80)
    print("✅ 데이터 수집 및 전처리 완료!")
    print("="*80)
    print()
    print("📊 최종 통계:")
    print("─"*80)
    print(f"  • 전체 데이터: {len(df_sorted)}건")
    print(f"  • 컬럼 수: {len(df_sorted.columns)}개")
    print(f"  • URL 컬럼: 포함 ✅")
    print()
    
    return df_sorted

print("✅ 함수 정의 완료: collect_all_notices_with_url()")


## 📋 Step 3: CSV 저장 함수

DataFrame을 CSV 파일로 저장하여 팀원들과 공유합니다.


In [ ]:
def save_notices_to_csv(df, filename=None, include_all_columns=True):
    """
    공고 데이터를 CSV 파일로 저장
    
    Parameters:
        df (DataFrame): 저장할 DataFrame
        filename (str): 파일명 (None이면 자동 생성)
        include_all_columns (bool): 전체 컬럼 포함 여부 (기본값: True)
    
    Returns:
        str: 저장된 파일 경로
    """
    if df is None or len(df) == 0:
        print("❌ 저장할 데이터가 없습니다")
        return None
    
    print("="*80)
    print("💾 CSV 파일 저장")
    print("="*80)
    print()
    
    # 파일명 자동 생성
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"GH_notices_{timestamp}.csv"
    
    # 파일명에 .csv 확장자가 없으면 추가
    if not filename.endswith('.csv'):
        filename = f"{filename}.csv"
    
    # output 디렉토리 생성
    output_dir = 'output'
    os.makedirs(output_dir, exist_ok=True)
    filepath = os.path.join(output_dir, filename)
    
    # 저장할 DataFrame 준비
    df_to_save = df.copy()
    
    # 주요 컬럼만 선택 (선택사항)
    if not include_all_columns:
        key_columns = [
            '공고번호', '공고명', '공고일자', '공고시각',
            '시군구명', '구분', '모집횟수',
            '접수시작일자', '접수종료일자',
            '당첨자발표일자', '입주예정년월',
            '접수처주소', '접수처전화번호',
            '첨부파일일련번호',
            'url'  # URL은 항상 포함
        ]
        
        # 존재하는 컬럼만 선택
        available_columns = [col for col in key_columns if col in df_to_save.columns]
        df_to_save = df_to_save[available_columns]
        print(f"📋 주요 컬럼만 저장: {len(available_columns)}개")
    else:
        print(f"📋 전체 컬럼 저장: {len(df_to_save.columns)}개")
    
    # CSV 저장 (UTF-8 with BOM for Excel compatibility)
    df_to_save.to_csv(filepath, index=False, encoding='utf-8-sig')
    
    # 파일 크기 확인
    file_size = os.path.getsize(filepath)
    file_size_mb = file_size / (1024 * 1024)
    
    print()
    print("="*80)
    print("✅ CSV 저장 완료!")
    print("="*80)
    print()
    print("📊 파일 정보:")
    print("─"*80)
    print(f"  • 파일명: {filename}")
    print(f"  • 경로: {filepath}")
    print(f"  • 데이터 수: {len(df_to_save):,}건")
    print(f"  • 컬럼 수: {len(df_to_save.columns)}개")
    print(f"  • 파일 크기: {file_size_mb:.2f} MB")
    print()
    print("💡 사용 방법:")
    print("─"*80)
    print("  • Excel에서 열기: 파일 더블클릭")
    print("  • Python에서 읽기: pd.read_csv('경로')")
    print("  • URL 컬럼: 가장 오른쪽에 위치")
    print()
    
    return filepath

print("✅ 함수 정의 완료: save_notices_to_csv()")


## 📋 Step 4: 전체 프로세스 실행

이제 모든 단계를 실행하여 2024-2025년 공고 데이터를 CSV로 저장합니다.


In [ ]:
# 🚀 전체 프로세스 실행
print("="*80)
print("🚀 2024-2025 공고 데이터 수집 및 CSV 저장 프로세스")
print("="*80)
print()

# Step 1: 데이터 수집
df_notices = collect_all_notices_with_url(min_year='2024', exclude_test=True)

if df_notices is not None and len(df_notices) > 0:
    print("="*80)
    print("📊 데이터 미리보기 (상위 10개)")
    print("="*80)
    print()
    
    # 주요 컬럼만 출력
    display_columns = ['공고번호', '공고명', '공고일자', '시군구명', 'url']
    available_display_columns = [col for col in display_columns if col in df_notices.columns]
    
    for idx, row in df_notices.head(10).iterrows():
        print(f"[{idx+1:2d}] ─────────────────────────────────────────────────────")
        
        # 공고명
        notice_name = row.get('공고명', 'N/A')
        if isinstance(notice_name, str) and len(notice_name) > 60:
            notice_name = notice_name[:60] + "..."
        print(f"     제목: {notice_name}")
        
        # 공고일자
        notice_date = str(row.get('공고일자', 'N/A'))
        if len(notice_date) == 8:
            notice_date_formatted = f"{notice_date[:4]}-{notice_date[4:6]}-{notice_date[6:]}"
        else:
            notice_date_formatted = notice_date
        print(f"     공고일: {notice_date_formatted}")
        
        # 공고번호
        print(f"     공고번호: {row.get('공고번호', 'N/A')}")
        
        # 지역
        region = row.get('시군구명', 'N/A')
        if pd.isna(region) or region == 'None':
            region = 'N/A'
        print(f"     지역: {region}")
        
        # URL
        print(f"     URL: {row.get('url', 'N/A')}")
        print()
    
    print("="*80)
    print()
    
    # Step 2: CSV 저장 (전체 컬럼 포함)
    print("💾 CSV 파일 저장 중...\n")
    csv_path = save_notices_to_csv(
        df_notices,
        filename='GH_notices_2024_2025',
        include_all_columns=True  # 전체 컬럼 포함
    )
    
    if csv_path:
        print("="*80)
        print("🎉 프로세스 완료!")
        print("="*80)
        print()
        print("📝 다음 단계:")
        print("─"*80)
        print("  1. output 폴더의 CSV 파일 확인")
        print("  2. 팀원들에게 파일 공유")
        print("  3. URL 컬럼을 활용하여 크롤링 진행")
        print()
        print("💡 URL 확인 방법:")
        print("─"*80)
        print("  • CSV 파일 열기")
        print("  • 맨 오른쪽 'url' 컬럼 확인")
        print("  • 각 URL을 브라우저에서 열어 공고 상세 페이지 확인")
        print()
        print("⚠️  주의사항:")
        print("─"*80)
        print("  • URL 패턴이 실제와 다를 수 있음 (수동 확인 필요)")
        print("  • GH 홈페이지에서 실제 공고 클릭하여 URL 형식 확인")
        print("  • 필요시 URL 패턴 수정")
        print()
        
        # DataFrame 변수 저장 안내
        print("💾 변수 저장:")
        print("─"*80)
        print("  • df_notices : 전체 공고 DataFrame")
        print(f"  • 데이터 크기: {df_notices.shape}")
        print()
else:
    print("❌ 데이터 수집 실패")
    print("   API 호출 또는 데이터 필터링에 문제가 있습니다")

print("="*80)


## 📋 Step 5: URL 패턴 검증 (선택 사항)

실제 GH 홈페이지의 URL 패턴과 비교하여 확인합니다.


In [ ]:
# URL 패턴 검증 (선택 사항)
print("="*80)
print("🔍 URL 패턴 검증")
print("="*80)
print()

if 'df_notices' in locals() and df_notices is not None and len(df_notices) > 0:
    print("📋 최신 5개 공고의 URL 샘플:")
    print("─"*80)
    print()
    
    for idx, row in df_notices.head(5).iterrows():
        notice_name = row.get('공고명', 'N/A')
        if isinstance(notice_name, str) and len(notice_name) > 50:
            notice_name = notice_name[:50] + "..."
        
        notice_no = row.get('공고번호', 'N/A')
        url = row.get('url', 'N/A')
        
        print(f"[{idx+1}] {notice_name}")
        print(f"    공고번호: {notice_no}")
        print(f"    생성된 URL: {url}")
        print()
    
    print("="*80)
    print("🔧 URL 패턴 수정이 필요한 경우")
    print("="*80)
    print()
    print("만약 생성된 URL이 실제와 다르다면:")
    print()
    print("1️⃣ GH 홈페이지에서 공고 클릭")
    print("   → https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseList.do")
    print()
    print("2️⃣ 브라우저 주소창의 URL 복사")
    print("   예: https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseDtl.do?pbancNo=742")
    print()
    print("3️⃣ URL 패턴 확인")
    print("   • 기본 URL: https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseDtl.do")
    print("   • 파라미터: ?pbancNo=공고번호")
    print("   • 추가 파라미터 있는지 확인")
    print()
    print("4️⃣ generate_notice_url 함수 수정 (필요시)")
    print("   예: 추가 파라미터가 필요한 경우")
    print()
    print("```python")
    print("def generate_notice_url(notice_number, additional_param=None):")
    print("    base_url = 'https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseDtl.do'")
    print("    url = f'{base_url}?pbancNo={notice_number}'")
    print("    if additional_param:")
    print("        url += f'&{additional_param}'")
    print("    return url")
    print("```")
    print()
    
    print("="*80)
    print("✅ URL 검증 완료!")
    print("="*80)
    print()
    print("💡 다음 작업:")
    print("  1. 위의 샘플 URL 중 하나를 브라우저에서 열어보기")
    print("  2. 공고 상세 페이지가 정상적으로 표시되는지 확인")
    print("  3. URL 패턴이 맞으면 크롤링 진행")
    print("  4. URL 패턴이 틀리면 함수 수정 후 재실행")
    print()
else:
    print("⚠️  먼저 Step 4를 실행하여 df_notices를 생성하세요")

print("="*80)


---

## 🎉 완료 요약

### ✅ 완료된 작업

1. **URL 패턴 분석**
   - GH 공고 상세 페이지 URL 패턴 정의
   - `generate_notice_url()` 함수 생성

2. **데이터 수집 함수**
   - `collect_all_notices_with_url()` 함수 생성
   - 2024-2025년 데이터 필터링
   - 중복 제거 (공고번호 + 공고일자 + 구분)
   - 테스트 공고 제외
   - URL 컬럼 자동 추가

3. **CSV 저장 함수**
   - `save_notices_to_csv()` 함수 생성
   - 전체 컬럼 또는 주요 컬럼 선택 가능
   - UTF-8 with BOM (Excel 호환)
   - URL 컬럼이 맨 오른쪽에 위치

4. **실행 및 검증**
   - 전체 프로세스 자동화
   - URL 샘플 출력 및 검증 가이드

### 📁 생성된 파일

```
output/
└── GH_notices_2024_2025.csv
    ├── 전체 컬럼 (62개 이상)
    └── url 컬럼 (맨 오른쪽)
```

### 🔧 사용 방법

#### 1. CSV 파일 확인
```python
import pandas as pd

# CSV 읽기
df = pd.read_csv('output/GH_notices_2024_2025.csv')

# 데이터 확인
print(df.shape)  # (행, 열)
print(df.columns)  # 컬럼 목록
print(df.head())  # 상위 5개
print(df['url'].head())  # URL 확인
```

#### 2. 팀원과 공유
- `output/GH_notices_2024_2025.csv` 파일 공유
- Excel에서 바로 열기 가능
- URL 컬럼(맨 오른쪽)을 활용하여 크롤링

#### 3. 크롤링 시작
```python
# 예시: Selenium 사용
from selenium import webdriver

for idx, row in df.iterrows():
    url = row['url']
    driver.get(url)
    # 크롤링 로직...
```

### ⚠️ 주의사항

1. **URL 패턴 검증 필수**
   - 생성된 URL이 실제 GH 홈페이지와 일치하는지 확인
   - Step 5에서 샘플 URL을 브라우저에서 테스트
   - 필요시 `generate_notice_url()` 함수 수정

2. **API 데이터 제한**
   - API 최신 데이터: 2025-07-31까지
   - 그 이후 데이터는 웹 스크래핑 필요

3. **중복 데이터**
   - 같은 공고번호에 다른 구분(유형)이 있을 수 있음
   - 이는 중복이 아닌 별도 공고 (예: 일반자, 예비자)

### 📚 다음 단계

1. **URL 패턴 확인**
   - [ ] Step 5 실행하여 샘플 URL 확인
   - [ ] 브라우저에서 URL 접속 테스트
   - [ ] 필요시 함수 수정

2. **크롤링 준비**
   - [ ] Selenium 또는 BeautifulSoup 설치
   - [ ] 크롤링 스크립트 작성
   - [ ] 첨부파일 다운로드 로직 추가

3. **데이터 통합**
   - [ ] LH API 데이터와 통합 (추후)
   - [ ] 데이터베이스 저장 (추후)

---


## 5. 실제 사용 예시

### 최신 10개 공고 조회 및 JSON 저장

In [8]:
# 최신 10개 공고 가져오기
result = get_latest_notices_for_app(count=10, exclude_test=True, min_year='2024')

if result['success']:
    print("="*80)
    print("📊 조회 결과")
    print("="*80)
    print()
    print(f"✅ 성공: {result['success']}")
    print(f"📅 API 마지막 업데이트: {result['last_api_update']}")
    print(f"📊 반환된 공고 수: {result['count']}건")
    print(f"💡 안내: {result['notice']}")
    print()
    
    # 공고 목록 출력 (최대 5개)
    print("="*80)
    print("📢 공고 목록 (최대 5개)")
    print("="*80)
    print()
    
    for idx, notice in enumerate(result['data'][:5], 1):
        date_str = str(notice.get('공고일자', 'N/A'))
        if len(date_str) == 8:
            date_formatted = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
        else:
            date_formatted = date_str
        
        print(f"[{idx}] {date_formatted} | {notice.get('공고명', '제목 없음')[:60]}")
        print(f"    공고번호: {notice.get('공고번호')} | 지역: {notice.get('시군구명', 'N/A')}")
        print()
    
    if result['count'] > 5:
        print(f"... 외 {result['count'] - 5}건")
        print()
    
    # JSON 파일로 내보내기
    filepath = export_latest_notices_json(result)
    
    print()
    print("="*80)
    print("✅ 앱/웹 연동 준비 완료!")
    print("="*80)
else:
    print("❌ 데이터 조회 실패")
    print(f"   에러: {result.get('error', '알 수 없는 오류')}")

📡 API 데이터 수집 중 (최소 연도: 2024)...
✅ 총 990건 수집 완료\n
🎯 최신 10개 추출 완료
📅 API 마지막 업데이트: 2025--0-7-31
💡 안내: 본 데이터는 2025--0-7-31 기준입니다. 최신 공고는 GH 홈페이지를 확인하세요.\n
📊 조회 결과

✅ 성공: True
📅 API 마지막 업데이트: 2025--0-7-31
📊 반환된 공고 수: 10건
💡 안내: 본 데이터는 2025--0-7-31 기준입니다. 최신 공고는 GH 홈페이지를 확인하세요.

📢 공고 목록 (최대 5개)

[1] 2025-07-31 | (최초) 광주역세권 청년혁신타운 통합공공임대주택(일자리연계형 지원주택) 입주자 모집 공고
    공고번호: 742 | 지역: N/A

[2] 2025-07-24 | 안성청사복합 통합공공임대주택 단지 내 공공임대상가 임차인 2차 모집 공고
    공고번호: 741 | 지역: N/A

[3] 2025-07-18 | (선착순수의계약) 안성청사복합 통합공공임대주택 단지 내 일반형 임대상가 임차인 모집 공고
    공고번호: 740 | 지역: N/A

[4] 2025-07-18 | (선착순수의계약) 용인영덕 경기행복주택 단지 내 일반형 임대상가 임차인 모집 공고
    공고번호: 739 | 지역: N/A

[5] 2025-07-18 | (선착순수의계약) 수원영통 경기행복주택 단지 내 일반형 임대상가 임차인 모집 공고
    공고번호: 738 | 지역: N/A

... 외 5건

✅ JSON 파일 저장 완료!
📁 경로: output\gh_notices_20251117_175253.json
📊 데이터: 10건

✅ 앱/웹 연동 준비 완료!


## 6. API Metadata 구조

### GH API 공고 데이터 필드 구조

#### 📋 기본 정보
```python
{
    "공고번호": 742,
    "공고명": "(최초) 광주역세권 청년혁신타운 통합공공임대주택...",
    "공고일자": "20250731",
    "공고시각": "17:00",
    "게시일자": "20250731",
    "게시시각": "17:00",
    "구분": "990",
    "모집횟수": 1
}
```

#### 📍 위치 정보
```python
{
    "시군구명": "광주시",
    "접수처주소": "경기도 광주시...",
    "접수처상세주소": "...",
    "접수처우편번호": "12345",
    "접수처전화번호": "031-XXX-XXXX"
}
```

#### 📆 일정 정보
```python
{
    "접수시작일자": "20250801",
    "접수시작시각": "10:00",
    "접수종료일자": "20250822",
    "접수종료시각": "18:00",
    "당첨자발표일자": "20250825",
    "당첨자발표시각": "10:00",
    "계약기간시작일자": "20250901",
    "계약기간종료일자": "20250915",
    "입주예정년월": "202512"
}
```

#### 🏠 주택 정보
```python
{
    "주택관리번호": "2025-XXXX",
    "사업코드": "XXXX"
}
```

#### 📎 첨부파일
```python
{
    "첨부파일일련번호": 12345,
    "제출서류첨부파일일련번호": 67890
}
```

### 필드 목록 (전체)

총 **60개 이상**의 필드가 제공됩니다:

| 카테고리 | 필드 수 | 주요 필드 |
|---------|--------|-----------|
| 기본 정보 | 8 | 공고번호, 공고명, 공고일자, 구분, 모집횟수 |
| 위치 정보 | 5 | 시군구명, 접수처주소, 접수처전화번호 |
| 일정 정보 | 20+ | 접수시작/종료, 발표일자, 계약기간, 입주예정년월 |
| 주택 정보 | 3 | 주택관리번호, 사업코드, 입주예정년월 |
| 첨부파일 | 2 | 첨부파일일련번호, 제출서류첨부파일일련번호 |
| 기타 정보 | 20+ | 유의사항, 비고, 정정사유, 계약특약내용 등 |



---

## 7. LH API 통합 준비

### 🎯 목표: GH + LH API 통합

**공통 Metadata 구조 설계**를 통해 하나의 앱에서 GH와 LH 공고를 통합 관리

### 1️⃣ API 비교

| 항목 | GH API | LH API |
|------|--------|--------|
| 제공기관 | 경기주택도시공사 | 한국토지주택공사 |
| 포함 데이터 | 경기도 지역 | 전국 (LH 관할) |
| 주택 유형 | 행복주택, 국민임대, 통합공공임대, 장기전세 | 국민임대, 영구임대, 행복주택, 분양주택 등 |
| 엔드포인트 | OpenAPI (공공데이터포털) | OpenAPI (공공데이터포털) |
| 업데이트 주기 | 월 1회? (추정) | ? (확인 필요) |

### 2️⃣ 통합 Metadata 설계

#### 공통 필드 (필수)

```python
{
    # 기본 정보
    "source": "GH" | "LH",  # 데이터 출처
    "notice_id": "unique_id",  # 공고 고유번호
    "notice_title": "공고명",
    "notice_date": "20250731",  # YYYYMMDD
    "notice_type": "행복주택" | "국민임대" | ...,
    
    # 위치 정보
    "region": "경기도" | "서울시" | ...,
    "city": "광주시",
    "address": "상세 주소",
    "postal_code": "12345",
    
    # 일정 정보
    "application_start": "20250801",
    "application_end": "20250822",
    "announcement_date": "20250825",  # 당첨자 발표일
    "contract_start": "20250901",
    "contract_end": "20250915",
    "move_in_date": "202512",  # YYYYMM
    
    # 연락처
    "contact_phone": "031-XXX-XXXX",
    
    # 첨부파일
    "attachments": [
        {
            "file_id": 12345,
            "file_name": "공고문.pdf",
            "file_url": "https://..."
        }
    ],
    
    # 메타데이터
    "created_at": "2025-07-31T17:00:00",
    "updated_at": "2025-07-31T17:00:00",
    "raw_data": {...}  # 원본 데이터 전체 저장
}
```

### 3️⃣ 통합 함수 설계 (예시)

```python
def normalize_gh_notice(gh_data):
    """GH API 데이터를 공통 포맷으로 변환"""
    return {
        "source": "GH",
        "notice_id": f"GH-{gh_data['공고번호']}",
        "notice_title": gh_data['공고명'],
        "notice_date": gh_data['공고일자'],
        "notice_type": gh_data.get('구분', 'N/A'),
        "region": "경기도",
        "city": gh_data.get('시군구명', 'N/A'),
        "address": gh_data.get('접수처주소', 'N/A'),
        "postal_code": gh_data.get('접수처우편번호', 'N/A'),
        "application_start": gh_data.get('접수시작일자', 'N/A'),
        "application_end": gh_data.get('접수종료일자', 'N/A'),
        "announcement_date": gh_data.get('당첨자발표일자', 'N/A'),
        "contract_start": gh_data.get('계약기간시작일자', 'N/A'),
        "contract_end": gh_data.get('계약기간종료일자', 'N/A'),
        "move_in_date": gh_data.get('입주예정년월', 'N/A'),
        "contact_phone": gh_data.get('접수처전화번호', 'N/A'),
        "attachments": [
            {
                "file_id": gh_data.get('첨부파일일련번호'),
                "file_name": "공고문",
                "file_url": None  # GH는 직접 다운로드 불가
            }
        ] if gh_data.get('첨부파일일련번호') else [],
        "created_at": f"{gh_data['공고일자'][:4]}-{gh_data['공고일자'][4:6]}-{gh_data['공고일자'][6:]}",
        "updated_at": f"{gh_data['공고일자'][:4]}-{gh_data['공고일자'][4:6]}-{gh_data['공고일자'][6:]}",
        "raw_data": gh_data
    }

def normalize_lh_notice(lh_data):
    """LH API 데이터를 공통 포맷으로 변환 (추후 구현)"""
    # LH API 구조에 맞춰 구현
    pass

def merge_notices(gh_notices, lh_notices):
    """GH + LH 공고 데이터 병합"""
    all_notices = []
    
    # GH 데이터 정규화
    for gh in gh_notices:
        all_notices.append(normalize_gh_notice(gh))
    
    # LH 데이터 정규화
    for lh in lh_notices:
        all_notices.append(normalize_lh_notice(lh))
    
    # 날짜순 정렬
    all_notices.sort(key=lambda x: x['notice_date'], reverse=True)
    
    return all_notices
```

### 4️⃣ 앱/웹 표시 통합

```javascript
// 앱/웹에서 공통 포맷 데이터 사용
function displayNotice(notice) {
    const source_badge = notice.source === 'GH' ? '🏢 GH' : '🏛️ LH';
    
    return `
        <div class="notice-card">
            <span class="badge">${source_badge}</span>
            <h3>${notice.notice_title}</h3>
            <p>📅 공고일: ${formatDate(notice.notice_date)}</p>
            <p>📍 지역: ${notice.region} ${notice.city}</p>
            <p>📆 접수: ${formatDate(notice.application_start)} ~ ${formatDate(notice.application_end)}</p>
            <p>📞 연락처: ${notice.contact_phone}</p>
            
            ${notice.attachments.length > 0 ? 
                '<button>📎 첨부파일 다운로드</button>' : 
                ''
            }
        </div>
    `;
}
```

---

## 8. 최종 권장사항

### ✅ GH API 활용 시 체크리스트

- [x] API 키 발급 완료
- [x] 2025 엔드포인트 사용
- [x] 최신 데이터 조회 함수 구현 (`get_latest_notices_for_app`)
- [x] JSON 내보내기 함수 구현 (`export_latest_notices_json`)
- [x] API 업데이트 지연 안내 메시지 포함
- [x] 홈페이지 링크 제공
- [ ] 정기 모니터링 설정 (월 1회 권장)
- [ ] LH API 연동 (추후)

### 🚀 다음 단계

1. **LH API 조사**
   - LH 공공데이터 포털에서 주택청약 API 확인
   - Metadata 구조 분석
   - GH API와 필드 매핑

2. **통합 함수 구현**
   - `normalize_gh_notice()` 완성
   - `normalize_lh_notice()` 구현
   - `merge_notices()` 테스트

3. **앱/웹 연동**
   - 공통 포맷 JSON 생성
   - 프론트엔드에서 통합 표시
   - 필터링 기능 (지역, 유형, 날짜)

4. **유지보수**
   - 월 1회 API 업데이트 확인
   - 새로운 필드 추가 시 통합 포맷 업데이트
   - 사용자 피드백 반영

---

## 📚 참고 자료

### 공공데이터 포털
- GH 주택청약 모집정보: https://www.data.go.kr/data/15119414/
- LH 주택청약 정보: https://www.data.go.kr/ (검색 필요)

### GH 홈페이지
- 주택청약센터: https://apply.gh.or.kr/sb/sr/sr7150/selectPbancRentHouseList.do

### API 문의
- 공공데이터 포털: 1600-4119
- GH 고객센터: 031-250-1000

---

## 🎉 완료!

이제 GH API를 활용하여 최신 공고 데이터를 수집하고, 앱/웹에 연동할 준비가 완료되었습니다!

**다음 작업**: LH API 통합 및 통합 포맷 구현

